Environment Check and Prerequisites
==========================================
Before we start, let's understand what we're building:

1. We have some mock financial data CSV files)
2. We need to move this data into a "real" database (PostgreSQL)
3. We'll learn SQL by querying our own financial data
4. This mirrors real FinTech companies' data pipelines

What you'll learn:
- How to connect Python to PostgreSQL
- How to import CSV files into database tables
- How to validate data quality
- How to write SQL queries for financial analysis

Prerequisites Check:
- PostgreSQL installed (version 14+)
- pgAdmin installed (comes with PostgreSQL)
- Python packages: pandas, numpy, psycopg2, sqlalchemy
- Your generated data in 'mock_financial_data' folder

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime

# Check Python version
print(f"🐍 Python version: {sys.version}")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")

# Check if we have the synthetic data 
data_dir = 'mock_financial_data'
if os.path.exists(data_dir):
    print(f"\n✅ Found data directory: {data_dir}")
    files = os.listdir(data_dir)
    print(f"📁 Files available: {len(files)}")
    for file in sorted(files):
        if file.endswith('.csv'):
            size_mb = os.path.getsize(os.path.join(data_dir, file)) / (1024*1024)
            print(f"   - {file}: {size_mb:.2f} MB")
else:
    print(f"\n❌ Data directory not found!")
    print("Please run Week 0 data generation first")
    print("Run: generator.save_all_datasets()")

🐍 Python version: 3.13.2 (tags/v3.13.2:4f8bb39, Feb  4 2025, 15:23:48) [MSC v.1942 64 bit (AMD64)]
📊 Pandas version: 2.2.3
🔢 NumPy version: 2.2.6

✅ Found data directory: mock_financial_data
📁 Files available: 7
   - crypto_prices.csv: 4.46 MB
   - customer_data.csv: 0.61 MB
   - economic_indicators.csv: 0.02 MB
   - portfolio_data.csv: 0.37 MB
   - stock_prices.csv: 1.28 MB


Installing Database Connection Libraries
===============================================
We need special Python packages to talk to PostgreSQL:

1. psycopg2: Low-level PostgreSQL adapter (like a translator)
2. sqlalchemy: High-level database toolkit (makes complex operations easier)

Think of it like:
- psycopg2 = manual transmission (more control, more complex)
- sqlalchemy = automatic transmission (easier to use, less control)

If installation fails:
- Windows: You might need Visual C++ build tools
- Mac: You might need to install PostgreSQL first
- Linux: You might need postgresql-dev package

In [2]:
import subprocess
import sys

def install_package(package_name):
    """Helper function to install packages"""
    try:
        __import__(package_name.replace('-', '_'))
        print(f"✅ {package_name} already installed")
    except ImportError:
        print(f"📦 Installing {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"✅ {package_name} installed successfully")

# Install required packages
packages_needed = [
    'psycopg2-binary',  # PostgreSQL adapter
    'sqlalchemy',       # SQL toolkit
    'python-dotenv'     # For secure password storage
]

print("🔧 Checking required packages...\n")
for package in packages_needed:
    install_package(package)

print("\n✅ All required packages are ready!")

🔧 Checking required packages...

📦 Installing psycopg2-binary...
✅ psycopg2-binary installed successfully
✅ sqlalchemy already installed
📦 Installing python-dotenv...
✅ python-dotenv installed successfully

✅ All required packages are ready!


Setting Up Database Connection
=====================================

IMPORTANT: Database connections are like phone calls:
1. You need the right "phone number" (host, port, database name)
2. You need to "authenticate" yourself (username, password)
3. The connection can "drop" if not used properly
4. Always "hang up" (close connection) when done

Security Note:
- NEVER hardcode passwords in your code
- NEVER commit passwords to GitHub
- Use environment variables or config files

Let's create a safe connection setup:

In [3]:
import psycopg2
from sqlalchemy import create_engine
import getpass  # For secure password input

# Database configuration class
class DatabaseConfig:
    """
    Stores database connection parameters.
    In production, these would come from environment variables.
    """
    def __init__(self):
        self.host = 'localhost'      # Your computer
        self.port = 5432            # Default PostgreSQL port
        self.database = 'fintech_db' # Database name we'll create
        self.user = 'postgres'       # Default PostgreSQL user
        
        # Secure password input
        print("🔐 PostgreSQL Connection Setup")
        print(f"Host: {self.host}")
        print(f"Port: {self.port}")
        print(f"Database: {self.database}")
        print(f"User: {self.user}")
        
        # Get password securely (won't show on screen)
        self.password = getpass.getpass("Enter PostgreSQL password: ")
        
    def get_connection_string(self):
        """Create connection string for SQLAlchemy"""
        return f"postgresql://{self.user}:{self.password}@{self.host}:{self.port}/{self.database}"
    
    def get_connection_params(self):
        """Get parameters for psycopg2"""
        return {
            'host': self.host,
            'port': self.port,
            'database': self.database,
            'user': self.user,
            'password': self.password
        }

# Create configuration
db_config = DatabaseConfig()

# Test connection
def test_connection(config):
    """Test if we can connect to PostgreSQL"""
    try:
        # First, connect to PostgreSQL server (not specific database)
        conn_params = config.get_connection_params()
        conn_params['database'] = 'postgres'  # Default database
        
        conn = psycopg2.connect(**conn_params)
        cur = conn.cursor()
        
        # Check PostgreSQL version
        cur.execute("SELECT version();")
        version = cur.fetchone()[0]
        print(f"\n✅ Connected to PostgreSQL!")
        print(f"📊 Version: {version.split(',')[0]}")
        
        cur.close()
        conn.close()
        return True
        
    except Exception as e:
        print(f"\n❌ Connection failed: {e}")
        print("\nTroubleshooting:")
        print("1. Is PostgreSQL running? Check pgAdmin")
        print("2. Is the password correct?")
        print("3. Is the port 5432 free?")
        return False

# Test the connection
if test_connection(db_config):
    print("\n🎉 Database connection successful!")
else:
    print("\n⚠️ Please fix connection issues before proceeding")

🔐 PostgreSQL Connection Setup
Host: localhost
Port: 5432
Database: fintech_db
User: postgres


Enter PostgreSQL password:  ········



✅ Connected to PostgreSQL!
📊 Version: PostgreSQL 17.5 on x86_64-windows

🎉 Database connection successful!


Creating Our FinTech Database
====================================

Databases are like filing cabinets:
- PostgreSQL server = The entire office
- Database = One filing cabinet  
- Tables = Drawers in the cabinet
- Rows = Individual files in drawers

We'll create a dedicated database for our FinTech project.
This keeps our data organized and separate from other projects.

Important Concepts:
- CREATE DATABASE: Makes a new database
- DROP DATABASE: Deletes a database (careful!)
- IF EXISTS / IF NOT EXISTS: Prevents errors

In [4]:
def create_fintech_database(config):
    """
    Create the fintech_db database if it doesn't exist.
    This is like creating a new filing cabinet for our project.
    """
    try:
        # Connect to PostgreSQL server (not a specific database)
        conn_params = config.get_connection_params()
        conn_params['database'] = 'postgres'  # Default database
        
        # Important: autocommit=True for CREATE DATABASE
        conn = psycopg2.connect(**conn_params)
        conn.autocommit = True  # Required for CREATE DATABASE
        cur = conn.cursor()
        
        # Check if database exists
        cur.execute("""
            SELECT 1 FROM pg_database WHERE datname = %s
        """, (config.database,))
        
        exists = cur.fetchone()
        
        if exists:
            print(f"📊 Database '{config.database}' already exists")
            
            # Get database size
            cur.execute(f"""
                SELECT pg_size_pretty(pg_database_size('{config.database}'))
            """)
            size = cur.fetchone()[0]
            print(f"📏 Current size: {size}")
            
        else:
            print(f"🏗️ Creating database '{config.database}'...")
            cur.execute(f"CREATE DATABASE {config.database}")
            print(f"✅ Database '{config.database}' created successfully!")
        
        # List all databases (for learning purposes)
        print("\n📋 All databases on this server:")
        cur.execute("""
            SELECT datname, pg_size_pretty(pg_database_size(datname)) as size
            FROM pg_database 
            WHERE datistemplate = false
            ORDER BY datname;
        """)
        
        for db_name, db_size in cur.fetchall():
            print(f"   - {db_name}: {db_size}")
        
        cur.close()
        conn.close()
        return True
        
    except Exception as e:
        print(f"❌ Error creating database: {e}")
        return False

# Create our database
if create_fintech_database(db_config):
    print("\n✅ Database setup complete!")
else:
    print("\n⚠️ Database creation failed")

🏗️ Creating database 'fintech_db'...
✅ Database 'fintech_db' created successfully!

📋 All databases on this server:
   - fintech_db: 7681 kB
   - postgres: 11 MB

✅ Database setup complete!


Designing Database Tables (Schemas)
==========================================

A table schema is like a blueprint for data storage.
Just like a form has specific fields, tables have columns with specific types.

Data Types in PostgreSQL:
- INTEGER: Whole numbers (-2B to +2B)
- BIGINT: Very large whole numbers
- NUMERIC(10,2): Decimal with 10 total digits, 2 after decimal
- VARCHAR(50): Text up to 50 characters
- TEXT: Unlimited text
- DATE: Calendar date (no time)
- TIMESTAMP: Date + time
- BOOLEAN: True/False

Primary Keys:
- Unique identifier for each row
- Like a social security number for data
- Can be one column or combination of columns

Indexes:
- Like a book's index - helps find data faster
- Trade-off: Faster reads, slower writes


In [5]:
# Let's examine what tables we need
def analyze_csv_structure(data_dir='mock_financial_data'):
    """
    Analyze our CSV files to understand what tables we need.
    This helps us design appropriate database schemas.
    """
    print("📊 Analyzing CSV files to design table schemas...\n")
    
    csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
    
    table_designs = {}
    
    for csv_file in csv_files:
        print(f"📄 Analyzing: {csv_file}")
        print("-" * 50)
        
        # Read first few rows to understand structure
        df = pd.read_csv(os.path.join(data_dir, csv_file), nrows=5)
        
        # Analyze columns
        print(f"Columns: {len(df.columns)}")
        print(f"Sample rows: {len(df)}")
        print("\nColumn Analysis:")
        
        table_name = csv_file.replace('.csv', '').replace('_data', 's')
        columns_info = []
        
        for col in df.columns:
            # Determine PostgreSQL data type
            dtype = str(df[col].dtype)
            sample_value = df[col].iloc[0] if len(df) > 0 else None
            
            if 'date' in col.lower() or 'time' in col.lower():
                if 'timestamp' in col.lower():
                    pg_type = 'TIMESTAMP'
                else:
                    pg_type = 'DATE'
            elif dtype == 'object':
                # Check if it's a string column
                max_len = df[col].astype(str).str.len().max()
                if max_len <= 50:
                    pg_type = f'VARCHAR({int(max_len * 1.5)})'  # Add buffer
                else:
                    pg_type = 'TEXT'
            elif 'int' in dtype:
                if df[col].max() > 2147483647:  # Max INT value
                    pg_type = 'BIGINT'
                else:
                    pg_type = 'INTEGER'
            elif 'float' in dtype:
                # Determine precision needed
                if 'price' in col.lower() or 'value' in col.lower():
                    pg_type = 'NUMERIC(20,8)'  # High precision for prices
                elif 'weight' in col.lower() or 'return' in col.lower():
                    pg_type = 'NUMERIC(7,4)'   # Percentages
                else:
                    pg_type = 'NUMERIC(20,4)'  # General decimal
            elif 'bool' in dtype:
                pg_type = 'BOOLEAN'
            else:
                pg_type = 'TEXT'  # Fallback
            
            columns_info.append({
                'name': col,
                'pandas_type': dtype,
                'postgres_type': pg_type,
                'sample': sample_value
            })
            
            print(f"  - {col:20} {dtype:10} → {pg_type:15} (sample: {sample_value})")
        
        table_designs[table_name] = columns_info
        print("\n")
    
    return table_designs

# Analyze our data
table_designs = analyze_csv_structure()
print("✅ Schema analysis complete!")

📊 Analyzing CSV files to design table schemas...

📄 Analyzing: crypto_prices.csv
--------------------------------------------------
Columns: 7
Sample rows: 5

Column Analysis:
  - Timestamp            object     → TIMESTAMP       (sample: 2020-01-01 00:00:00)
  - Symbol               object     → VARCHAR(4)      (sample: BTC)
  - Open                 float64    → NUMERIC(20,4)   (sample: 37445.66)
  - High                 float64    → NUMERIC(20,4)   (sample: 37641.73)
  - Low                  float64    → NUMERIC(20,4)   (sample: 37445.66)
  - Close                float64    → NUMERIC(20,4)   (sample: 37607.94)
  - Volume               int64      → INTEGER         (sample: 513543)


📄 Analyzing: customer_data.csv
--------------------------------------------------
Columns: 11
Sample rows: 5

Column Analysis:
  - CustomerID           object     → VARCHAR(16)     (sample: CUST_000001)
  - Age                  int64      → INTEGER         (sample: 66)
  - Income               float64    →

Cell 6: Creating Tables in PostgreSQL
====================================

Now we'll create the actual tables in our database.
This is like setting up the drawers in our filing cabinet.

SQL Commands we'll use:
- CREATE TABLE: Makes a new table
- PRIMARY KEY: Unique identifier for rows
- NOT NULL: This column cannot be empty
- CREATE INDEX: Speed up searches

Best Practices:
1. Always include created_at timestamp
2. Use meaningful column names
3. Choose appropriate data types
4. Add indexes for frequently searched columns


In [6]:
def create_tables(config):
    """
    Create all necessary tables for our FinTech project.
    Each table is designed for specific financial data.
    """
    
    # Table definitions with explanations
    tables = {
        'stock_prices': {
            'description': 'Daily stock price data (OHLCV format)',
            'sql': """
                CREATE TABLE IF NOT EXISTS stock_prices (
                    -- Primary key columns (unique identifier)
                    date DATE NOT NULL,
                    symbol VARCHAR(10) NOT NULL,
                    
                    -- Price data (OHLCV)
                    open NUMERIC(10,2),      -- Opening price
                    high NUMERIC(10,2),      -- Highest price of day
                    low NUMERIC(10,2),       -- Lowest price of day
                    close NUMERIC(10,2),     -- Closing price
                    volume BIGINT,           -- Number of shares traded
                    
                    -- Metadata
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    
                    -- Composite primary key (date + symbol must be unique)
                    PRIMARY KEY (date, symbol)
                );
                
                -- Indexes for faster queries
                CREATE INDEX IF NOT EXISTS idx_stock_symbol ON stock_prices(symbol);
                CREATE INDEX IF NOT EXISTS idx_stock_date ON stock_prices(date);
                
                -- Add comments for documentation
                COMMENT ON TABLE stock_prices IS 'Daily stock price data in OHLCV format';
                COMMENT ON COLUMN stock_prices.symbol IS 'Stock ticker symbol (e.g., AAPL, GOOGL)';
                COMMENT ON COLUMN stock_prices.volume IS 'Number of shares traded during the day';
            """
        },
        
        'crypto_prices': {
            'description': '6-hourly cryptocurrency price data',
            'sql': """
                CREATE TABLE IF NOT EXISTS crypto_prices (
                    -- Crypto trades 24/7, so we use timestamp
                    timestamp TIMESTAMP NOT NULL,
                    symbol VARCHAR(10) NOT NULL,
                    
                    -- Price data (higher precision for crypto)
                    open NUMERIC(20,8),      -- 8 decimals for small altcoins
                    high NUMERIC(20,8),
                    low NUMERIC(20,8),
                    close NUMERIC(20,8),
                    volume BIGINT,
                    
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    PRIMARY KEY (timestamp, symbol)
                );
                
                CREATE INDEX IF NOT EXISTS idx_crypto_symbol ON crypto_prices(symbol);
                CREATE INDEX IF NOT EXISTS idx_crypto_timestamp ON crypto_prices(timestamp);
                
                COMMENT ON TABLE crypto_prices IS '6-hourly cryptocurrency price data';
                COMMENT ON COLUMN crypto_prices.timestamp IS 'UTC timestamp of the price snapshot';
            """
        },
        
        'economic_indicators': {
            'description': 'Monthly macroeconomic indicators',
            'sql': """
                CREATE TABLE IF NOT EXISTS economic_indicators (
                    date DATE NOT NULL,
                    indicator VARCHAR(50) NOT NULL,
                    value NUMERIC(20,4),
                    unit VARCHAR(20),        -- %, USD, points, etc.
                    
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    PRIMARY KEY (date, indicator)
                );
                
                CREATE INDEX IF NOT EXISTS idx_econ_indicator ON economic_indicators(indicator);
                CREATE INDEX IF NOT EXISTS idx_econ_date ON economic_indicators(date);
                
                COMMENT ON TABLE economic_indicators IS 'Monthly macroeconomic indicators';
                COMMENT ON COLUMN economic_indicators.indicator IS 'Indicator name (e.g., GDP_GROWTH, INFLATION_RATE)';
                COMMENT ON COLUMN economic_indicators.unit IS 'Unit of measurement';
            """
        },
        
        'portfolio_holdings': {
            'description': 'Monthly portfolio snapshots',
            'sql': """
                CREATE TABLE IF NOT EXISTS portfolio_holdings (
                    date DATE NOT NULL,
                    portfolio_id VARCHAR(20) NOT NULL,
                    
                    -- Portfolio characteristics
                    risk_level VARCHAR(20),   -- Conservative/Moderate/Aggressive
                    total_value NUMERIC(20,2),
                    
                    -- Asset allocation (must sum to 1.0)
                    stock_weight NUMERIC(5,4),  -- 0.0000 to 1.0000
                    bond_weight NUMERIC(5,4),
                    cash_weight NUMERIC(5,4),
                    
                    -- Performance
                    monthly_return NUMERIC(7,4),  -- -0.9999 to 9.9999
                    
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    PRIMARY KEY (date, portfolio_id)
                );
                
                CREATE INDEX IF NOT EXISTS idx_portfolio_id ON portfolio_holdings(portfolio_id);
                CREATE INDEX IF NOT EXISTS idx_portfolio_risk ON portfolio_holdings(risk_level);
                
                COMMENT ON TABLE portfolio_holdings IS 'Monthly portfolio performance and allocation data';
            """
        },
        
        'customers': {
            'description': 'Customer demographics and account information',
            'sql': """
                CREATE TABLE IF NOT EXISTS customers (
                    -- Customer identity
                    customer_id VARCHAR(20) PRIMARY KEY,
                    
                    -- Demographics
                    age INTEGER CHECK (age >= 18 AND age <= 120),
                    income NUMERIC(12,2) CHECK (income >= 0),
                    credit_score INTEGER CHECK (credit_score >= 300 AND credit_score <= 850),
                    
                    -- Account information
                    account_age_days INTEGER CHECK (account_age_days >= 0),
                    account_balance NUMERIC(20,2),
                    
                    -- Behavior metrics
                    monthly_transactions INTEGER CHECK (monthly_transactions >= 0),
                    avg_transaction_amount NUMERIC(12,2),
                    num_products INTEGER CHECK (num_products >= 0),
                    has_loan BOOLEAN,
                    
                    -- Risk classification
                    risk_segment VARCHAR(20),
                    
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                );
                
                CREATE INDEX IF NOT EXISTS idx_customer_risk ON customers(risk_segment);
                CREATE INDEX IF NOT EXISTS idx_customer_credit ON customers(credit_score);
                
                COMMENT ON TABLE customers IS 'Customer profiles for analytics and risk assessment';
                COMMENT ON COLUMN customers.credit_score IS 'FICO credit score (300-850 range)';
            """
        }
    }
    
    try:
        # Connect to our fintech database
        conn = psycopg2.connect(**config.get_connection_params())
        cur = conn.cursor()
        
        print("🏗️ Creating database tables...\n")
        
        for table_name, table_info in tables.items():
            print(f"📊 Creating table: {table_name}")
            print(f"   Description: {table_info['description']}")
            
            # Execute the CREATE TABLE statement
            cur.execute(table_info['sql'])
            
            # Check if table was created successfully
            cur.execute("""
                SELECT COUNT(*) 
                FROM information_schema.tables 
                WHERE table_schema = 'public' 
                AND table_name = %s
            """, (table_name,))
            
            if cur.fetchone()[0] > 0:
                print(f"   ✅ Table created successfully")
                
                # Get column count
                cur.execute("""
                    SELECT COUNT(*) 
                    FROM information_schema.columns 
                    WHERE table_name = %s
                """, (table_name,))
                col_count = cur.fetchone()[0]
                print(f"   📋 Columns: {col_count}")
            else:
                print(f"   ❌ Table creation failed")
            
            print()
        
        # Commit all changes
        conn.commit()
        
        # Show summary of created tables
        print("\n📊 Database Schema Summary:")
        print("-" * 60)
        cur.execute("""
            SELECT 
                table_name,
                COUNT(*) as column_count
            FROM information_schema.columns
            WHERE table_schema = 'public'
            GROUP BY table_name
            ORDER BY table_name;
        """)
        
        for table, col_count in cur.fetchall():
            print(f"Table: {table:20} Columns: {col_count}")
        
        cur.close()
        conn.close()
        print("\n✅ All tables created successfully!")
        return True
        
    except Exception as e:
        print(f"❌ Error creating tables: {e}")
        if conn:
            conn.rollback()
        return False

# Create all tables
if create_tables(db_config):
    print("🎉 Database schema is ready!")
else:
    print("⚠️ Please fix table creation errors")

🏗️ Creating database tables...

📊 Creating table: stock_prices
   Description: Daily stock price data (OHLCV format)
   ✅ Table created successfully
   📋 Columns: 8

📊 Creating table: crypto_prices
   Description: 6-hourly cryptocurrency price data
   ✅ Table created successfully
   📋 Columns: 8

📊 Creating table: economic_indicators
   Description: Monthly macroeconomic indicators
   ✅ Table created successfully
   📋 Columns: 5

📊 Creating table: portfolio_holdings
   Description: Monthly portfolio snapshots
   ✅ Table created successfully
   📋 Columns: 9

📊 Creating table: customers
   Description: Customer demographics and account information
   ✅ Table created successfully
   📋 Columns: 12


📊 Database Schema Summary:
------------------------------------------------------------
Table: crypto_prices        Columns: 8
Table: customers            Columns: 12
Table: economic_indicators  Columns: 5
Table: portfolio_holdings   Columns: 9
Table: stock_prices         Columns: 8

✅ All tabl

Understanding the Data Import Process
============================================

Before we import data, let's understand what we're doing:

CSV Files → PostgreSQL Tables

Challenges:
1. Data types might not match
2. Dates need special formatting
3. NULL values need handling
4. Large files need batch processing

Methods for importing data:
1. COPY command (fastest, but less flexible)
2. INSERT statements (slow, but most control)
3. pandas.to_sql (balanced approach)

We'll use pandas.to_sql because:
- It handles data type conversion
- It can append to existing tables
- It provides progress feedback
- It's easier to debug

In [7]:
def preview_csv_data(csv_file, data_dir='mock_financial_data', rows=5):
    """
    Preview CSV data before import to spot potential issues.
    This is like checking your ingredients before cooking.
    """
    filepath = os.path.join(data_dir, csv_file)
    
    print(f"📄 Previewing: {csv_file}")
    print("=" * 60)
    
    # Read first few rows
    df = pd.read_csv(filepath, nrows=rows)
    
    # Basic statistics
    total_rows = sum(1 for line in open(filepath)) - 1  # Subtract header
    
    print(f"Total rows: {total_rows:,}")
    print(f"Columns: {list(df.columns)}")
    print(f"\nFirst {rows} rows:")
    print(df)
    
    # Check for potential issues
    print(f"\nData Quality Checks:")
    
    # Check for nulls
    null_counts = df.isnull().sum()
    if null_counts.any():
        print("⚠️ Found NULL values:")
        print(null_counts[null_counts > 0])
    else:
        print("✅ No NULL values in sample")
    
    # Check data types
    print(f"\nData types:")
    for col, dtype in df.dtypes.items():
        print(f"  {col}: {dtype}")
    
    # Memory usage
    memory_usage = df.memory_usage(deep=True).sum() / 1024**2
    estimated_total = memory_usage * (total_rows / rows)
    print(f"\nEstimated memory needed: {estimated_total:.2f} MB")
    
    return df

# Preview each CSV file
csv_files = [
    'stock_prices.csv',
    'crypto_prices.csv', 
    'economic_indicators.csv',
    'portfolio_data.csv',
    'customer_data.csv'
]

print("🔍 Previewing all CSV files before import...\n")

for csv_file in csv_files:
    if os.path.exists(os.path.join('mock_financial_data', csv_file)):
        preview_df = preview_csv_data(csv_file)
        print("\n" + "-"*60 + "\n")
        # Wait for user to review
        input("Press Enter to continue to next file...")

🔍 Previewing all CSV files before import...

📄 Previewing: stock_prices.csv
Total rows: 26,100
Columns: ['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'Volume']

First 5 rows:
         Date Symbol    Open    High     Low   Close    Volume
0  2020-01-01   AAPL  362.42  364.30  362.42  364.09   4598546
1  2020-01-02   AAPL  365.91  365.91  357.94  357.98    367620
2  2020-01-03   AAPL  356.91  356.91  352.20  353.07  29745876
3  2020-01-06   AAPL  353.57  353.57  345.71  349.30  10824733
4  2020-01-07   AAPL  349.35  355.84  349.35  355.05   1553500

Data Quality Checks:
✅ No NULL values in sample

Data types:
  Date: object
  Symbol: object
  Open: float64
  High: float64
  Low: float64
  Close: float64
  Volume: int64

Estimated memory needed: 4.44 MB

------------------------------------------------------------



Press Enter to continue to next file... 


📄 Previewing: crypto_prices.csv
Total rows: 73,050
Columns: ['Timestamp', 'Symbol', 'Open', 'High', 'Low', 'Close', 'Volume']

First 5 rows:
             Timestamp Symbol      Open      High       Low     Close  Volume
0  2020-01-01 00:00:00    BTC  37445.66  37641.73  37445.66  37607.94  513543
1  2020-01-01 06:00:00    BTC  38265.57  38918.27  37993.31  38353.88   14045
2  2020-01-01 12:00:00    BTC  39178.44  39178.44  36829.42  38377.20   20102
3  2020-01-01 18:00:00    BTC  38204.61  38471.59  38198.89  38264.98  282156
4  2020-01-02 00:00:00    BTC  39139.10  39139.10  35926.36  37411.67   76198

Data Quality Checks:
✅ No NULL values in sample

Data types:
  Timestamp: object
  Symbol: object
  Open: float64
  High: float64
  Low: float64
  Close: float64
  Volume: int64

Estimated memory needed: 12.99 MB

------------------------------------------------------------



Press Enter to continue to next file... 


📄 Previewing: economic_indicators.csv
Total rows: 600
Columns: ['Date', 'Indicator', 'Value']

First 5 rows:
         Date            Indicator  Value
0  2020-01-31           GDP_GROWTH   2.39
1  2020-01-31       INFLATION_RATE   1.95
2  2020-01-31    UNEMPLOYMENT_RATE   5.10
3  2020-01-31        INTEREST_RATE   1.31
4  2020-01-31  CONSUMER_CONFIDENCE  94.59

Data Quality Checks:
✅ No NULL values in sample

Data types:
  Date: object
  Indicator: object
  Value: float64

Estimated memory needed: 0.09 MB

------------------------------------------------------------



Press Enter to continue to next file... 


📄 Previewing: portfolio_data.csv
Total rows: 6,000
Columns: ['Date', 'PortfolioID', 'RiskLevel', 'TotalValue', 'StockWeight', 'BondWeight', 'CashWeight', 'MonthlyReturn']

First 5 rows:
         Date PortfolioID RiskLevel  TotalValue  StockWeight  BondWeight  \
0  2020-01-31      PF_001  Moderate  1242670.88        0.569       0.382   
1  2020-02-29      PF_001  Moderate  1297316.35        0.567       0.383   
2  2020-03-31      PF_001  Moderate  1212010.39        0.562       0.388   
3  2020-04-30      PF_001  Moderate  1225080.79        0.566       0.383   
4  2020-05-31      PF_001  Moderate  1191190.49        0.578       0.372   

   CashWeight  MonthlyReturn  
0       0.049         0.0273  
1       0.050         0.0440  
2       0.050        -0.0658  
3       0.050         0.0108  
4       0.050        -0.0277  

Data Quality Checks:
✅ No NULL values in sample

Data types:
  Date: object
  PortfolioID: object
  RiskLevel: object
  TotalValue: float64
  StockWeight: float64
  BondW

Press Enter to continue to next file... 


📄 Previewing: customer_data.csv
Total rows: 10,000
Columns: ['CustomerID', 'Age', 'Income', 'CreditScore', 'AccountAgeDays', 'AccountBalance', 'MonthlyTransactions', 'AvgTransactionAmount', 'NumProducts', 'HasLoan', 'RiskSegment']

First 5 rows:
    CustomerID  Age    Income  CreditScore  AccountAgeDays  AccountBalance  \
0  CUST_000001   66  20000.00          621             764         1111.06   
1  CUST_000002   41  36815.04          590            1789       110834.08   
2  CUST_000003   26  52834.47          748            1853          466.99   
3  CUST_000004   62  56594.88          693            1176          190.74   
4  CUST_000005   23  52210.34          578            1238         5254.55   

   MonthlyTransactions  AvgTransactionAmount  NumProducts  HasLoan RiskSegment  
0                   17                 26.79            2    False      Medium  
1                   54                 72.19            1    False        High  
2                   61                124.

Press Enter to continue to next file... 


Importing Stock Price Data
==================================

Let's start with stock prices - our most important dataset.
We'll import it step by step with proper error handling.

Key considerations:
1. Date formatting (pandas → PostgreSQL)
2. Handling duplicates
3. Transaction management
4. Progress tracking


In [8]:
from sqlalchemy import create_engine
import pandas as pd

def import_stock_prices(config, data_dir='mock_financial_data'):
    """
    Import stock price data with detailed logging.
    This is our most critical dataset.
    """
    csv_file = 'stock_prices.csv'
    filepath = os.path.join(data_dir, csv_file)
    
    print(f"📈 Importing Stock Price Data")
    print("=" * 60)
    
    try:
        # Step 1: Load the CSV file
        print("Step 1: Loading CSV file...")
        df = pd.read_csv(filepath)
        print(f"✅ Loaded {len(df):,} rows")
        
        # Step 2: Data preprocessing
        print("\nStep 2: Preprocessing data...")
        
        # Convert date column to datetime
        df['date'] = pd.to_datetime(df['Date'])
        df = df.drop('Date', axis=1)  # Remove original column
        
        # Ensure column names match database
        df.columns = [col.lower() for col in df.columns]
        
        # Check for duplicates
        duplicates = df.duplicated(subset=['date', 'symbol'])
        if duplicates.any():
            print(f"⚠️ Found {duplicates.sum()} duplicate rows")
            df = df[~duplicates]
            print(f"✅ Removed duplicates, {len(df):,} rows remaining")
        
        # Data validation
        print("\nStep 3: Validating data...")
        
        # Check OHLC logic (High >= Low, etc.)
        invalid_ohlc = df[
            (df['high'] < df['low']) |
            (df['high'] < df['open']) |
            (df['high'] < df['close']) |
            (df['low'] > df['open']) |
            (df['low'] > df['close'])
        ]
        
        if len(invalid_ohlc) > 0:
            print(f"⚠️ Found {len(invalid_ohlc)} rows with invalid OHLC")
            print("First few invalid rows:")
            print(invalid_ohlc.head())
        else:
            print("✅ All OHLC values are valid")
        
        # Check for negative prices
        negative_prices = df[(df[['open', 'high', 'low', 'close']] < 0).any(axis=1)]
        if len(negative_prices) > 0:
            print(f"⚠️ Found {len(negative_prices)} rows with negative prices")
        else:
            print("✅ No negative prices found")
        
        # Step 4: Import to database
        print("\nStep 4: Importing to PostgreSQL...")
        
        # Create SQLAlchemy engine
        engine = create_engine(config.get_connection_string())
        
        # Import in chunks for better performance
        chunk_size = 5000
        total_chunks = len(df) // chunk_size + 1
        
        for i in range(0, len(df), chunk_size):
            chunk = df.iloc[i:i+chunk_size]
            chunk.to_sql(
                'stock_prices',
                engine,
                if_exists='append',
                index=False,
                method='multi'
            )
            
            # Progress update
            current_chunk = i // chunk_size + 1
            progress = (i + len(chunk)) / len(df) * 100
            print(f"  Chunk {current_chunk}/{total_chunks}: {progress:.1f}% complete")
        
        print(f"\n✅ Successfully imported {len(df):,} stock price records")
        
        # Step 5: Verify import
        print("\nStep 5: Verifying import...")
        
        conn = psycopg2.connect(**config.get_connection_params())
        cur = conn.cursor()
        
        # Count imported rows
        cur.execute("SELECT COUNT(*) FROM stock_prices")
        db_count = cur.fetchone()[0]
        
        # Get date range
        cur.execute("""
            SELECT MIN(date), MAX(date), COUNT(DISTINCT symbol)
            FROM stock_prices
        """)
        min_date, max_date, symbol_count = cur.fetchone()
        
        print(f"Database summary:")
        print(f"  Total rows: {db_count:,}")
        print(f"  Date range: {min_date} to {max_date}")
        print(f"  Unique symbols: {symbol_count}")
        
        # Sample data
        print("\nSample data from database:")
        cur.execute("""
            SELECT date, symbol, open, high, low, close, volume
            FROM stock_prices
            ORDER BY date DESC, symbol
            LIMIT 5
        """)
        
        for row in cur.fetchall():
            print(f"  {row}")
        
        cur.close()
        conn.close()
        
        return True
        
    except Exception as e:
        print(f"\n❌ Import failed: {e}")
        print("\nTroubleshooting:")
        print("1. Check if table exists")
        print("2. Check column names match")
        print("3. Check date format")
        print("4. Check for disk space")
        return False

# Import stock prices
if import_stock_prices(db_config):
    print("\n🎉 Stock price import complete!")
else:
    print("\n⚠️ Please check stock import errors")

📈 Importing Stock Price Data
Step 1: Loading CSV file...
✅ Loaded 26,100 rows

Step 2: Preprocessing data...

Step 3: Validating data...
✅ All OHLC values are valid
✅ No negative prices found

Step 4: Importing to PostgreSQL...
  Chunk 1/6: 19.2% complete
  Chunk 2/6: 38.3% complete
  Chunk 3/6: 57.5% complete
  Chunk 4/6: 76.6% complete
  Chunk 5/6: 95.8% complete
  Chunk 6/6: 100.0% complete

✅ Successfully imported 26,100 stock price records

Step 5: Verifying import...
Database summary:
  Total rows: 26,100
  Date range: 2020-01-01 to 2024-12-31
  Unique symbols: 20

Sample data from database:
  (datetime.date(2024, 12, 31), 'AAPL', Decimal('421.30'), Decimal('425.83'), Decimal('421.30'), Decimal('425.73'), 1346229)
  (datetime.date(2024, 12, 31), 'AMZN', Decimal('354.89'), Decimal('360.83'), Decimal('354.89'), Decimal('360.01'), 16759835)
  (datetime.date(2024, 12, 31), 'BAC', Decimal('361.71'), Decimal('367.41'), Decimal('358.00'), Decimal('365.12'), 3052714)
  (datetime.date(202

Importing Cryptocurrency Price Data
==========================================

Cryptocurrency data has special characteristics:
1. Timestamp instead of date (24/7 trading)
2. Higher price precision (8 decimal places)
3. More volatile (larger price swings)
4. Different symbols (BTC, ETH vs AAPL, GOOGL)

Let's handle these differences properly.

In [9]:
def import_crypto_prices(config, data_dir='mock_financial_data'):
    """
    Import cryptocurrency price data with special handling
    for timestamps and high precision values.
    """
    csv_file = 'crypto_prices.csv'
    filepath = os.path.join(data_dir, csv_file)
    
    print(f"💎 Importing Cryptocurrency Price Data")
    print("=" * 60)
    
    try:
        # Step 1: Load CSV
        print("Step 1: Loading crypto data...")
        df = pd.read_csv(filepath)
        print(f"✅ Loaded {len(df):,} rows")
        
        # Show sample to understand structure
        print("\nSample data:")
        print(df.head(3))
        
        # Step 2: Preprocessing
        print("\nStep 2: Preprocessing crypto data...")
        
        # Convert timestamp and ensure lowercase columns
        df['timestamp'] = pd.to_datetime(df['Timestamp'])
        df = df.drop('Timestamp', axis=1)
        df.columns = [col.lower() for col in df.columns]
        
        # Check timestamp frequency (should be 6-hourly)
        time_diffs = df.groupby('symbol')['timestamp'].diff()
        most_common_diff = time_diffs.mode()[0]
        print(f"Most common time interval: {most_common_diff}")
        
        # Crypto-specific validation
        print("\nStep 3: Crypto-specific validation...")
        
        # Check for extreme volatility (>50% in 6 hours)
        df['price_change_pct'] = df.groupby('symbol')['close'].pct_change()
        extreme_moves = df[df['price_change_pct'].abs() > 0.5]
        
        if len(extreme_moves) > 0:
            print(f"⚠️ Found {len(extreme_moves)} extreme price moves (>50%)")
            print("Largest moves:")
            print(extreme_moves.nlargest(5, 'price_change_pct')[['timestamp', 'symbol', 'price_change_pct']])
        
        # Remove temporary column
        df = df.drop('price_change_pct', axis=1)
        
        # Check price precision
        print("\nChecking price precision needs:")
        for symbol in df['symbol'].unique()[:5]:  # Check first 5 symbols
            symbol_data = df[df['symbol'] == symbol]
            max_decimals = symbol_data['close'].apply(lambda x: len(str(x).split('.')[-1])).max()
            avg_price = symbol_data['close'].mean()
            print(f"  {symbol}: avg price ${avg_price:.2f}, max decimals: {max_decimals}")
        
        # Step 4: Import to database
        print("\nStep 4: Importing to PostgreSQL...")
        
        engine = create_engine(config.get_connection_string())
        
        # Import in chunks (crypto data is larger)
        chunk_size = 10000
        total_rows = len(df)
        
        for i in range(0, total_rows, chunk_size):
            chunk = df.iloc[i:i+chunk_size]
            
            # Progress bar
            progress = min((i + chunk_size) / total_rows * 100, 100)
            bar_length = 30
            filled = int(bar_length * progress / 100)
            bar = '█' * filled + '░' * (bar_length - filled)
            print(f"\r  Progress: [{bar}] {progress:.1f}%", end='')
            
            chunk.to_sql(
                'crypto_prices',
                engine,
                if_exists='append',
                index=False,
                method='multi'
            )
        
        print(f"\n✅ Imported {len(df):,} crypto price records")
        
        # Step 5: Verify and analyze
        print("\nStep 5: Verification and analysis...")
        
        conn = psycopg2.connect(**config.get_connection_params())
        cur = conn.cursor()
        
        # Get summary statistics
        cur.execute("""
            SELECT 
                symbol,
                COUNT(*) as data_points,
                MIN(timestamp) as first_data,
                MAX(timestamp) as last_data,
                AVG(close) as avg_price,
                STDDEV(close) as price_volatility
            FROM crypto_prices
            GROUP BY symbol
            ORDER BY avg_price DESC
        """)
        
        print("\nCrypto Summary by Symbol:")
        print(f"{'Symbol':>6} {'Points':>7} {'First Data':>20} {'Last Data':>20} {'Avg Price':>12} {'Volatility':>12}")
        print("-" * 100)
        
        for row in cur.fetchall():
            symbol, points, first, last, avg_price, volatility = row
            print(f"{symbol:>6} {points:>7,} {str(first):>20} {str(last):>20} ${avg_price:>11,.2f} ${volatility:>11,.2f}")
        
        cur.close()
        conn.close()
        
        return True
        
    except Exception as e:
        print(f"\n❌ Crypto import failed: {e}")
        return False

# Import crypto data
if import_crypto_prices(db_config):
    print("\n🎉 Crypto price import complete!")
else:
    print("\n⚠️ Please check crypto import errors")

💎 Importing Cryptocurrency Price Data
Step 1: Loading crypto data...
✅ Loaded 73,050 rows

Sample data:
             Timestamp Symbol      Open      High       Low     Close  Volume
0  2020-01-01 00:00:00    BTC  37445.66  37641.73  37445.66  37607.94  513543
1  2020-01-01 06:00:00    BTC  38265.57  38918.27  37993.31  38353.88   14045
2  2020-01-01 12:00:00    BTC  39178.44  39178.44  36829.42  38377.20   20102

Step 2: Preprocessing crypto data...
Most common time interval: 0 days 06:00:00

Step 3: Crypto-specific validation...

Checking price precision needs:
  BTC: avg price $25457.43, max decimals: 2
  ETH: avg price $1339.72, max decimals: 2
  BNB: avg price $479.32, max decimals: 2
  XRP: avg price $1.10, max decimals: 6
  ADA: avg price $0.41, max decimals: 6

Step 4: Importing to PostgreSQL...
  Progress: [██████████████████████████████] 100.0%
✅ Imported 73,050 crypto price records

Step 5: Verification and analysis...

Crypto Summary by Symbol:
Symbol  Points           First

 Importing Economic Indicators
=====================================

Economic indicators are different from price data:
1. Monthly frequency (not daily)
2. Different units (%, billions, index points)
3. Some can be negative (trade balance)
4. Wide value ranges (0.5% to millions)

Understanding the indicators:
- GDP_GROWTH: Quarterly GDP growth rate (%)
- INFLATION_RATE: Monthly CPI change (%)
- UNEMPLOYMENT_RATE: Monthly unemployment (%)
- INTEREST_RATE: Federal funds rate (%)
- etc.

In [10]:
def import_economic_indicators(config, data_dir='mock_financial_data'):
    """
    Import economic indicators with proper unit handling.
    """
    csv_file = 'economic_indicators.csv'
    filepath = os.path.join(data_dir, csv_file)
    
    print(f"🏛️ Importing Economic Indicators")
    print("=" * 60)
    
    try:
        # Load data
        print("Step 1: Loading economic data...")
        df = pd.read_csv(filepath)
        print(f"✅ Loaded {len(df):,} rows")
        
        # Understand the structure
        print("\nUnique indicators:")
        indicators = df['Indicator'].unique()
        for ind in indicators:
            ind_data = df[df['Indicator'] == ind]
            print(f"  {ind}: {len(ind_data)} observations, "
                  f"range: {ind_data['Value'].min():.2f} to {ind_data['Value'].max():.2f}")
        
        # Preprocessing
        print("\nStep 2: Preprocessing...")
        df['date'] = pd.to_datetime(df['Date'])
        df = df.drop('Date', axis=1)
        df.columns = [col.lower() for col in df.columns]
        
        # Add unit information based on indicator type
        unit_mapping = {
            'GDP_GROWTH': '%',
            'INFLATION_RATE': '%',
            'UNEMPLOYMENT_RATE': '%',
            'INTEREST_RATE': '%',
            'CONSUMER_CONFIDENCE': 'index',
            'RETAIL_SALES': '%',
            'INDUSTRIAL_PRODUCTION': '%',
            'HOUSING_STARTS': 'thousands',
            'TRADE_BALANCE': 'millions USD',
            'MONEY_SUPPLY': 'billions USD'
        }
        
        df['unit'] = df['indicator'].map(unit_mapping)
        
        # Validate economic logic
        print("\nStep 3: Economic validation...")
        
        # Check for unrealistic values
        validations = {
            'GDP_GROWTH': (-10, 20),          # Realistic GDP growth range
            'INFLATION_RATE': (-5, 30),       # Deflation to hyperinflation
            'UNEMPLOYMENT_RATE': (0, 30),     # 0% to depression levels
            'INTEREST_RATE': (-2, 20),        # Negative to very high rates
            'CONSUMER_CONFIDENCE': (0, 200),  # Index range
        }
        
        for indicator, (min_val, max_val) in validations.items():
            ind_data = df[df['indicator'] == indicator]
            out_of_range = ind_data[(ind_data['value'] < min_val) | (ind_data['value'] > max_val)]
            if len(out_of_range) > 0:
                print(f"⚠️ {indicator}: {len(out_of_range)} values outside expected range [{min_val}, {max_val}]")
        
        # Import to database
        print("\nStep 4: Importing to PostgreSQL...")
        engine = create_engine(config.get_connection_string())
        
        df.to_sql(
            'economic_indicators',
            engine,
            if_exists='append',
            index=False,
            method='multi'
        )
        
        print(f"✅ Imported {len(df):,} economic indicator records")
        
        # Verify and create a summary view
        print("\nStep 5: Creating economic summary view...")
        
        conn = psycopg2.connect(**config.get_connection_params())
        cur = conn.cursor()
        
        # Create a pivot view for easier analysis
        cur.execute("""
            CREATE OR REPLACE VIEW economic_indicators_pivot AS
            SELECT 
                date,
                MAX(CASE WHEN indicator = 'GDP_GROWTH' THEN value END) as gdp_growth,
                MAX(CASE WHEN indicator = 'INFLATION_RATE' THEN value END) as inflation_rate,
                MAX(CASE WHEN indicator = 'UNEMPLOYMENT_RATE' THEN value END) as unemployment_rate,
                MAX(CASE WHEN indicator = 'INTEREST_RATE' THEN value END) as interest_rate,
                MAX(CASE WHEN indicator = 'CONSUMER_CONFIDENCE' THEN value END) as consumer_confidence
            FROM economic_indicators
            WHERE indicator IN ('GDP_GROWTH', 'INFLATION_RATE', 'UNEMPLOYMENT_RATE', 
                               'INTEREST_RATE', 'CONSUMER_CONFIDENCE')
            GROUP BY date
            ORDER BY date DESC;
        """)
        
        print("✅ Created pivot view for easier analysis")
        
        # Show recent economic snapshot
        cur.execute("""
            SELECT * FROM economic_indicators_pivot
            LIMIT 5
        """)
        
        print("\nRecent Economic Snapshot:")
        columns = [desc[0] for desc in cur.description]
        print(f"{' | '.join(col[:12].ljust(12) for col in columns)}")
        print("-" * (13 * len(columns)))
        
        for row in cur.fetchall():
            formatted_row = []
            for i, val in enumerate(row):
                if i == 0:  # Date
                    formatted_row.append(str(val)[:12].ljust(12))
                else:  # Numeric values
                    formatted_row.append(f"{val:>12.2f}" if val else " " * 12)
            print(" | ".join(formatted_row))
        
        cur.close()
        conn.close()
        
        return True
        
    except Exception as e:
        print(f"\n❌ Economic indicators import failed: {e}")
        return False

# Import economic data
if import_economic_indicators(db_config):
    print("\n🎉 Economic indicators import complete!")
else:
    print("\n⚠️ Please check economic import errors")

🏛️ Importing Economic Indicators
Step 1: Loading economic data...
✅ Loaded 600 rows

Unique indicators:
  GDP_GROWTH: 60 observations, range: 1.46 to 3.59
  INFLATION_RATE: 60 observations, range: 1.29 to 4.00
  UNEMPLOYMENT_RATE: 60 observations, range: 3.11 to 6.04
  INTEREST_RATE: 60 observations, range: 0.29 to 1.76
  CONSUMER_CONFIDENCE: 60 observations, range: 85.65 to 119.37
  RETAIL_SALES: 60 observations, range: -0.71 to 3.66
  INDUSTRIAL_PRODUCTION: 60 observations, range: -0.72 to 3.03
  HOUSING_STARTS: 60 observations, range: 1024707.47 to 1381179.57
  TRADE_BALANCE: 60 observations, range: -111225.35 to -27215.41
  MONEY_SUPPLY: 60 observations, range: 15399.62 to 21076.51

Step 2: Preprocessing...

Step 3: Economic validation...

Step 4: Importing to PostgreSQL...
✅ Imported 600 economic indicator records

Step 5: Creating economic summary view...
✅ Created pivot view for easier analysis

Recent Economic Snapshot:
date         | gdp_growth   | inflation_ra | unemployment 

Importing Portfolio Holdings Data
=========================================

Portfolio data represents:
- Investment accounts with different risk profiles
- Monthly snapshots of holdings
- Asset allocation (stocks, bonds, cash)
- Performance metrics

Key validations:
- Weights must sum to 1.0 (100%)
- Returns should be realistic
- Total value should be positive

In [11]:
def import_portfolio_data(config, data_dir='mock_financial_data'):
    """
    Import portfolio holdings with allocation validation.
    """
    csv_file = 'portfolio_data.csv'
    filepath = os.path.join(data_dir, csv_file)
    
    print(f"💼 Importing Portfolio Holdings Data")
    print("=" * 60)
    
    try:
        # Load data
        print("Step 1: Loading portfolio data...")
        df = pd.read_csv(filepath)
        print(f"✅ Loaded {len(df):,} rows")
        
        # Basic preprocessing
        print("\nStep 2: Preprocessing...")
        df['date'] = pd.to_datetime(df['Date'])
        df = df.drop('Date', axis=1)
        df.columns = [col.lower() for col in df.columns]
        
        # Rename columns to match database schema
        column_mapping = {
            'portfolioid': 'portfolio_id',
            'risklevel': 'risk_level',
            'totalvalue': 'total_value',
            'stockweight': 'stock_weight',
            'bondweight': 'bond_weight',
            'cashweight': 'cash_weight',
            'monthlyreturn': 'monthly_return'
        }
        df = df.rename(columns=column_mapping)
        
        # Portfolio-specific validations
        print("\nStep 3: Portfolio validation...")
        
        # Check if weights sum to 1.0
        df['weight_sum'] = df['stock_weight'] + df['bond_weight'] + df['cash_weight']
        weight_errors = df[abs(df['weight_sum'] - 1.0) > 0.001]  # Allow small rounding errors
        
        if len(weight_errors) > 0:
            print(f"⚠️ Found {len(weight_errors)} rows where weights don't sum to 1.0")
            print("Sample errors:")
            print(weight_errors[['portfolio_id', 'date', 'weight_sum']].head())
            
            # Fix by normalizing
            print("Fixing weight allocations...")
            for idx in weight_errors.index:
                total = df.loc[idx, 'weight_sum']
                if total > 0:
                    df.loc[idx, 'stock_weight'] /= total
                    df.loc[idx, 'bond_weight'] /= total
                    df.loc[idx, 'cash_weight'] /= total
        
        df = df.drop('weight_sum', axis=1)
        
        # Analyze performance by risk level
        print("\nPerformance Analysis by Risk Level:")
        risk_stats = df.groupby('risk_level').agg({
            'monthly_return': ['mean', 'std', 'min', 'max'],
            'portfolio_id': 'nunique'
        })
        
        print(risk_stats)
        
        # Check for unrealistic returns (>100% or <-50% monthly)
        extreme_returns = df[(df['monthly_return'] > 1.0) | (df['monthly_return'] < -0.5)]
        if len(extreme_returns) > 0:
            print(f"\n⚠️ Found {len(extreme_returns)} extreme monthly returns")
            
        # Import to database
        print("\nStep 4: Importing to PostgreSQL...")
        engine = create_engine(config.get_connection_string())
        
        df.to_sql(
            'portfolio_holdings',
            engine,
            if_exists='append',
            index=False,
            method='multi',
            chunksize=5000
        )
        
        print(f"✅ Imported {len(df):,} portfolio records")
        
        # Create useful analysis view
        print("\nStep 5: Creating portfolio analysis views...")
        
        conn = psycopg2.connect(**config.get_connection_params())
        cur = conn.cursor()
        
        # Portfolio performance summary view
        cur.execute("""
            CREATE OR REPLACE VIEW portfolio_performance_summary AS
            WITH returns_calc AS (
                SELECT 
                    portfolio_id,
                    risk_level,
                    AVG(monthly_return) * 12 as annual_return,
                    STDDEV(monthly_return) * SQRT(12) as annual_volatility,
                    MIN(total_value) as min_value,
                    MAX(total_value) as max_value,
                    MAX(total_value) / MIN(total_value) - 1 as total_growth
                FROM portfolio_holdings
                GROUP BY portfolio_id, risk_level
            )
            SELECT 
                risk_level,
                COUNT(*) as portfolio_count,
                AVG(annual_return) as avg_annual_return,
                AVG(annual_volatility) as avg_annual_volatility,
                AVG(annual_return) / AVG(annual_volatility) as avg_sharpe_ratio,
                AVG(total_growth) as avg_total_growth
            FROM returns_calc
            GROUP BY risk_level
            ORDER BY 
                CASE risk_level 
                    WHEN 'Conservative' THEN 1 
                    WHEN 'Moderate' THEN 2 
                    WHEN 'Aggressive' THEN 3 
                END;
        """)
        
        print("✅ Created portfolio performance summary view")
        
        # Show the summary
        cur.execute("SELECT * FROM portfolio_performance_summary")
        
        print("\nPortfolio Performance Summary:")
        columns = [desc[0] for desc in cur.description]
        print(f"{' | '.join(col[:15].ljust(15) for col in columns)}")
        print("-" * (16 * len(columns)))
        
        for row in cur.fetchall():
            formatted_row = []
            for i, val in enumerate(row):
                if i == 0:  # Risk level
                    formatted_row.append(str(val)[:15].ljust(15))
                elif i == 1:  # Count
                    formatted_row.append(f"{val:>15d}")
                else:  # Numeric values
                    formatted_row.append(f"{val:>15.4f}")
            print(" | ".join(formatted_row))
        
        cur.close()
        conn.close()
        
        return True
        
    except Exception as e:
        print(f"\n❌ Portfolio import failed: {e}")
        return False

# Import portfolio data
if import_portfolio_data(db_config):
    print("\n🎉 Portfolio data import complete!")
else:
    print("\n⚠️ Please check portfolio import errors")

💼 Importing Portfolio Holdings Data
Step 1: Loading portfolio data...
✅ Loaded 6,000 rows

Step 2: Preprocessing...

Step 3: Portfolio validation...
⚠️ Found 826 rows where weights don't sum to 1.0
Sample errors:
   portfolio_id       date  weight_sum
3        PF_001 2020-04-30       0.999
9        PF_001 2020-10-31       1.001
15       PF_001 2021-04-30       1.001
26       PF_001 2022-03-31       0.999
38       PF_001 2023-03-31       0.999
Fixing weight allocations...

Performance Analysis by Risk Level:
             monthly_return                           portfolio_id
                       mean       std     min     max      nunique
risk_level                                                        
Aggressive         0.009203  0.057741 -0.1745  0.1898           21
Conservative       0.004643  0.026346 -0.0817  0.0942           35
Moderate           0.006048  0.038861 -0.1171  0.1410           44

Step 4: Importing to PostgreSQL...
✅ Imported 6,000 portfolio records

Step 5: Creat

Importing Customer Demographics Data
============================================

Customer data is sensitive and requires:
1. Privacy considerations (even for mock data)
2. Data type validation (age, credit scores)
3. Business rule validation
4. Risk segmentation accuracy

This data supports:
- Credit scoring models
- Customer segmentation
- Risk assessment
- Marketing analytics

In [12]:
def import_customer_data(config, data_dir='mock_financial_data'):
    """
    Import customer data with privacy and validation checks.
    """
    csv_file = 'customer_data.csv'
    filepath = os.path.join(data_dir, csv_file)
    
    print(f"👥 Importing Customer Demographics Data")
    print("=" * 60)
    
    try:
        # Load data
        print("Step 1: Loading customer data...")
        df = pd.read_csv(filepath)
        print(f"✅ Loaded {len(df):,} customer records")
        
        # Preprocessing
        print("\nStep 2: Preprocessing...")
        
        # Ensure column names match database
        df.columns = [col.lower().replace(' ', '_') for col in df.columns]
        
        # Convert boolean column
        df['has_loan'] = df['hasloan']
        df = df.drop('hasloan', axis=1)
        
        # Customer data validation
        print("\nStep 3: Customer data validation...")
        
        # Age validation (18-120)
        invalid_age = df[(df['age'] < 18) | (df['age'] > 120)]
        if len(invalid_age) > 0:
            print(f"⚠️ Found {len(invalid_age)} customers with invalid age")
        
        # Credit score validation (300-850)
        invalid_credit = df[(df['creditscore'] < 300) | (df['creditscore'] > 850)]
        if len(invalid_credit) > 0:
            print(f"⚠️ Found {len(invalid_credit)} customers with invalid credit scores")
        
        # Income validation (positive)
        negative_income = df[df['income'] < 0]
        if len(negative_income) > 0:
            print(f"⚠️ Found {len(negative_income)} customers with negative income")
        
        # Analyze customer segments
        print("\nCustomer Segmentation Analysis:")
        segment_stats = df.groupby('risksegment').agg({
            'customerid': 'count',
            'age': 'mean',
            'income': 'mean',
            'creditscore': 'mean',
            'accountbalance': 'mean',
            'has_loan': 'mean'
        }).round(2)
        
        segment_stats.columns = ['count', 'avg_age', 'avg_income', 'avg_credit', 'avg_balance', 'loan_rate']
        print(segment_stats)
        
        # Check for suspicious patterns
        print("\nData Quality Checks:")
        
        # Check for duplicate customer IDs
        duplicate_ids = df[df.duplicated('customerid', keep=False)]
        if len(duplicate_ids) > 0:
            print(f"⚠️ Found {len(duplicate_ids)} duplicate customer IDs")
        else:
            print("✅ No duplicate customer IDs")
        
        # Check correlation between income and credit score
        income_credit_corr = df['income'].corr(df['creditscore'])
        print(f"Income-Credit Score correlation: {income_credit_corr:.3f}")
        
        # Fix column names for database
        column_mapping = {
            'customerid': 'customer_id',
            'creditscore': 'credit_score',
            'accountagedays': 'account_age_days',
            'accountbalance': 'account_balance',
            'monthlytransactions': 'monthly_transactions',
            'avgtransactionamount': 'avg_transaction_amount',
            'numproducts': 'num_products',
            'risksegment': 'risk_segment'
        }
        df = df.rename(columns=column_mapping)
        
        # Import to database
        print("\nStep 4: Importing to PostgreSQL...")
        engine = create_engine(config.get_connection_string())
        
        # Import in chunks due to size
        chunk_size = 2000
        total_chunks = len(df) // chunk_size + 1
        
        for i in range(0, len(df), chunk_size):
            chunk = df.iloc[i:i+chunk_size]
            chunk.to_sql(
                'customers',
                engine,
                if_exists='append',
                index=False,
                method='multi'
            )
            
            current_chunk = i // chunk_size + 1
            print(f"  Imported chunk {current_chunk}/{total_chunks}")
        
        print(f"✅ Imported {len(df):,} customer records")
        
        # Create analysis views
        print("\nStep 5: Creating customer analysis views...")
        
        conn = psycopg2.connect(**config.get_connection_params())
        cur = conn.cursor()
        
        # Customer segmentation view
        cur.execute("""
            CREATE OR REPLACE VIEW customer_segments AS
            SELECT 
                risk_segment,
                COUNT(*) as customer_count,
                AVG(age) as avg_age,
                AVG(income) as avg_income,
                AVG(credit_score) as avg_credit_score,
                AVG(account_balance) as avg_balance,
                AVG(monthly_transactions) as avg_monthly_transactions,
                SUM(CASE WHEN has_loan THEN 1 ELSE 0 END)::FLOAT / COUNT(*) as loan_penetration,
                PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY income) as income_q1,
                PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY income) as income_median,
                PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY income) as income_q3
            FROM customers
            GROUP BY risk_segment;
        """)
        
        # High-value customer view
        cur.execute("""
            CREATE OR REPLACE VIEW high_value_customers AS
            SELECT 
                customer_id,
                age,
                income,
                credit_score,
                account_balance,
                num_products,
                risk_segment
            FROM customers
            WHERE account_balance > (SELECT PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY account_balance) FROM customers)
               OR income > (SELECT PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY income) FROM customers)
            ORDER BY account_balance DESC;
        """)
        
        print("✅ Created customer analysis views")
        
        # Show segment summary
        cur.execute("SELECT * FROM customer_segments ORDER BY risk_segment")
        
        print("\nCustomer Segment Summary:")
        columns = [desc[0] for desc in cur.description]
        
        # Print header
        print("\n" + "-" * 120)
        print(f"{'Risk':^12} | {'Count':^8} | {'Age':^6} | {'Income':^10} | {'Credit':^7} | {'Balance':^10} | {'Trans/Mo':^9} | {'Loan %':^7}")
        print("-" * 120)
        
        for row in cur.fetchall():
            risk, count, age, income, credit, balance, trans, loan_pct = row[:8]
            print(f"{risk:^12} | {count:^8,} | {age:^6.1f} | ${income:^9,.0f} | {credit:^7.0f} | ${balance:^9,.0f} | {trans:^9.1f} | {loan_pct*100:^6.1f}%")
        
        cur.close()
        conn.close()
        
        return True
        
    except Exception as e:
        print(f"\n❌ Customer import failed: {e}")
        return False

# Import customer data
if import_customer_data(db_config):
    print("\n🎉 Customer data import complete!")
else:
    print("\n⚠️ Please check customer import errors")

👥 Importing Customer Demographics Data
Step 1: Loading customer data...
✅ Loaded 10,000 customer records

Step 2: Preprocessing...

Step 3: Customer data validation...

Customer Segmentation Analysis:
             count  avg_age  avg_income  avg_credit  avg_balance  loan_rate
risksegment                                                                
High          1754    39.88    38930.80      546.63      6749.45       0.00
Low           2855    39.70    44284.21      804.70      9378.98       0.05
Medium        5391    39.85    41118.21      676.82      7221.16       0.02

Data Quality Checks:
✅ No duplicate customer IDs
Income-Credit Score correlation: 0.090

Step 4: Importing to PostgreSQL...
  Imported chunk 1/6
  Imported chunk 2/6
  Imported chunk 3/6
  Imported chunk 4/6
  Imported chunk 5/6
✅ Imported 10,000 customer records

Step 5: Creating customer analysis views...
✅ Created customer analysis views

Customer Segment Summary:

-----------------------------------------------

Comprehensive Data Validation
=====================================

Now that all data is imported, let's run comprehensive
validation queries to ensure data integrity across tables.

This includes:
1. Referential integrity checks
2. Data completeness analysis
3. Anomaly detection
4. Cross-table consistency

In [31]:
def run_data_validation(config):
    """
    Run comprehensive validation queries across all tables.
    Generate a data quality report.
    """
    print(f"🔍 Running Comprehensive Data Validation")
    print("=" * 60)
    
    try:
        conn = psycopg2.connect(**config.get_connection_params())
        cur = conn.cursor()
        
        validation_results = []
        
        # 1. Table Row Counts
        print("\n1. Table Row Counts:")
        print("-" * 40)
        
        tables = ['stock_prices', 'crypto_prices', 'economic_indicators', 
                  'portfolio_holdings', 'customers']
        
        for table in tables:
            cur.execute(f"SELECT COUNT(*) FROM {table}")
            count = cur.fetchone()[0]
            print(f"  {table:20}: {count:>10,} rows")
            validation_results.append({
                'check': f'{table} row count',
                'result': count,
                'status': 'PASS' if count > 0 else 'FAIL'
            })
        
        # 2. Date Range Consistency
        print("\n2. Date Range Analysis:")
        print("-" * 40)
        
        # Stock prices date range
        cur.execute("""
            SELECT MIN(date) as min_date, MAX(date) as max_date,
                   COUNT(DISTINCT date) as trading_days
            FROM stock_prices
        """)
        stock_dates = cur.fetchone()
        print(f"  Stock prices: {stock_dates[0]} to {stock_dates[1]} ({stock_dates[2]} days)")
        
        # Economic indicators date range
        cur.execute("""
            SELECT MIN(date) as min_date, MAX(date) as max_date,
                   COUNT(DISTINCT date) as months
            FROM economic_indicators
        """)
        econ_dates = cur.fetchone()
        print(f"  Economic data: {econ_dates[0]} to {econ_dates[1]} ({econ_dates[2]} months)")
        
        # 3. Missing Data Analysis
        print("\n3. Missing Data Check:")
        print("-" * 40)
        
        # Check for missing trading days (excluding weekends)
        cur.execute("""
            WITH expected_days AS (
                SELECT generate_series(
                    (SELECT MIN(date) FROM stock_prices),
                    (SELECT MAX(date) FROM stock_prices),
                    '1 day'::interval
                )::date AS trading_date
            ),
            actual_days AS (
                SELECT DISTINCT date FROM stock_prices
            )
            SELECT COUNT(*) as missing_days
            FROM expected_days e
            LEFT JOIN actual_days a ON e.trading_date = a.date
            WHERE a.date IS NULL
              AND EXTRACT(DOW FROM e.trading_date) NOT IN (0, 6)
        """)
        
        missing_days = cur.fetchone()[0]
        print(f"  Missing trading days: {missing_days}")
        validation_results.append({
            'check': 'Missing trading days',
            'result': missing_days,
            'status': 'PASS' if missing_days < 10 else 'WARNING'
        })
        
        # 4. Data Integrity Checks
        print("\n4. Data Integrity Checks:")
        print("-" * 40)
        
        # Check OHLC consistency
        cur.execute("""
            SELECT COUNT(*) as invalid_ohlc
            FROM stock_prices
            WHERE high < low 
               OR high < open 
               OR high < close
               OR low > open 
               OR low > close
        """)
        invalid_ohlc = cur.fetchone()[0]
        print(f"  Invalid OHLC relationships: {invalid_ohlc}")
        validation_results.append({
            'check': 'OHLC consistency',
            'result': invalid_ohlc,
            'status': 'PASS' if invalid_ohlc == 0 else 'FAIL'
        })
        
        # Check portfolio weight sums
        cur.execute("""
            SELECT COUNT(*) as invalid_weights
            FROM portfolio_holdings
            WHERE ABS((stock_weight + bond_weight + cash_weight) - 1.0) > 0.01
        """)
        invalid_weights = cur.fetchone()[0]
        print(f"  Invalid portfolio weights: {invalid_weights}")
        validation_results.append({
            'check': 'Portfolio weight sums',
            'result': invalid_weights,
            'status': 'PASS' if invalid_weights == 0 else 'FAIL'
        })
        
        # 5. Statistical Anomaly Detection
        print("\n5. Statistical Anomaly Detection:")
        print("-" * 40)
        
        # Check for extreme returns
        cur.execute("""
            WITH daily_returns AS (
                SELECT 
                    symbol,
                    date,
                    (close / LAG(close) OVER (PARTITION BY symbol ORDER BY date) - 1) as return
                FROM stock_prices
            )
            SELECT 
                COUNT(*) as extreme_returns,
                MAX(ABS(return)) as max_return
            FROM daily_returns
            WHERE ABS(return) > 0.20  -- 20% daily move
        """)
        result = cur.fetchone()
        extreme_returns, max_return = result[0], result[1] if result[1] is not None else 0
        print(f"  Extreme daily returns (>20%): {extreme_returns}")
        print(f"  Maximum daily return: {max_return*100:.1f}%")
        
        # 6. Cross-Table Consistency
        print("\n6. Cross-Table Consistency:")
        print("-" * 40)
        
        # Check date overlap between tables
        cur.execute("""
            WITH stock_dates AS (
                SELECT MIN(date) as min_date, MAX(date) as max_date FROM stock_prices
            ),
            econ_dates AS (
                SELECT MIN(date) as min_date, MAX(date) as max_date FROM economic_indicators
            )
            SELECT 
                CASE 
                    WHEN s.min_date <= e.min_date AND s.max_date >= e.max_date THEN 'Full overlap'
                    WHEN s.max_date < e.min_date OR s.min_date > e.max_date THEN 'No overlap'
                    ELSE 'Partial overlap'
                END as date_overlap
            FROM stock_dates s, econ_dates e
        """)
        date_overlap = cur.fetchone()[0]
        print(f"  Stock/Economic date overlap: {date_overlap}")
        
        # 7. Generate Quality Score
        print("\n7. Data Quality Score:")
        print("-" * 40)
        
        total_checks = len(validation_results)
        passed_checks = sum(1 for r in validation_results if r['status'] == 'PASS')
        quality_score = (passed_checks / total_checks) * 100 if total_checks > 0 else 0
        
        print(f"  Total checks: {total_checks}")
        print(f"  Passed: {passed_checks}")
        print(f"  Failed: {total_checks - passed_checks}")
        print(f"  Quality Score: {quality_score:.1f}%")
        
        # Generate detailed report
        report_content = f"""
# Data Quality Report
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Summary
- Quality Score: {quality_score:.1f}%
- Total Checks: {total_checks}
- Passed: {passed_checks}
- Failed: {total_checks - passed_checks}

## Detailed Results
"""
        
        for result in validation_results:
            status_emoji = "PASS" if result['status'] == 'PASS' else "FAIL"
            report_content += f"- {result['check']}: {result['result']} [{status_emoji}]\n"
        
        # Save report with UTF-8 encoding
        with open('data_quality_report.md', 'w', encoding='utf-8') as f:
            f.write(report_content)
        
        print(f"\n📄 Detailed report saved to: data_quality_report.md")
        
        cur.close()
        conn.close()
        
        return validation_results
        
    except Exception as e:
        print(f"❌ Validation error: {e}")
        return []

# Run validation
validation_results = run_data_validation(db_config)

🔍 Running Comprehensive Data Validation

1. Table Row Counts:
----------------------------------------
  stock_prices        :     26,100 rows
  crypto_prices       :     73,050 rows
  economic_indicators :        600 rows
  portfolio_holdings  :      6,000 rows
  customers           :     10,000 rows

2. Date Range Analysis:
----------------------------------------
  Stock prices: 2020-01-01 to 2024-12-31 (1305 days)
  Economic data: 2020-01-31 to 2024-12-31 (60 months)

3. Missing Data Check:
----------------------------------------
  Missing trading days: 0

4. Data Integrity Checks:
----------------------------------------
  Invalid OHLC relationships: 0
  Invalid portfolio weights: 0

5. Statistical Anomaly Detection:
----------------------------------------
  Extreme daily returns (>20%): 0
  Maximum daily return: 0.0%

6. Cross-Table Consistency:
----------------------------------------
  Stock/Economic date overlap: Full overlap

7. Data Quality Score:
-------------------------

Creating Analysis Views and Performance Indexes
=======================================================

Views are like "saved queries" that simplify complex analysis.
Indexes make queries faster by creating lookup tables.

We'll create views for:
1. Daily returns calculation
2. Moving averages
3. Volatility metrics
4. Correlation matrices
5. Portfolio performance

In [35]:
"""
Cell 14: Complete Analysis Infrastructure Setup (FIXED)
======================================================

This version fixes:
1. The "cannot drop columns from view" error
2. The ROUND function type casting issue
3. The correlation_matrix view/table conflict

All analysis infrastructure in one robust solution.
"""

def create_complete_analysis_infrastructure_fixed(config):
    """
    Create all analysis infrastructure with proper error handling and fixes.
    """
    print("🏗️ Creating Complete Analysis Infrastructure")
    print("=" * 60)
    
    # Initialize results tracking
    results = {
        'views': {'created': [], 'failed': []},
        'indexes': {'created': [], 'failed': []},
        'correlations': {'status': 'pending', 'count': 0}
    }
    
    # PART 0: CLEANUP (if needed)
    print("\n🧹 PART 0: Cleaning Up Existing Objects")
    print("-" * 40)
    
    try:
        conn = psycopg2.connect(**config.get_connection_params())
        conn.autocommit = True
        cur = conn.cursor()
        
        # Drop the problematic correlation_matrix if it exists
        cur.execute("DROP VIEW IF EXISTS correlation_matrix CASCADE")
        cur.execute("DROP MATERIALIZED VIEW IF EXISTS correlation_matrix CASCADE")
        print("✅ Cleaned up existing correlation_matrix")
        
        cur.close()
        conn.close()
    except Exception as e:
        print(f"⚠️ Cleanup note: {e}")
    
    # PART 1: CREATE VIEWS
    print("\n📊 PART 1: Creating Analysis Views")
    print("-" * 40)
    
    views = {
        'daily_returns': """
            CREATE OR REPLACE VIEW daily_returns AS
            WITH price_data AS (
                SELECT 
                    date,
                    symbol,
                    close,
                    LAG(close) OVER (PARTITION BY symbol ORDER BY date) as prev_close
                FROM stock_prices
            )
            SELECT 
                date,
                symbol,
                close,
                prev_close,
                CASE 
                    WHEN prev_close IS NOT NULL AND prev_close > 0 
                    THEN (close - prev_close) / prev_close 
                    ELSE NULL 
                END as simple_return,
                CASE 
                    WHEN prev_close IS NOT NULL AND prev_close > 0 
                    THEN LN(close / prev_close) 
                    ELSE NULL 
                END as log_return
            FROM price_data
            ORDER BY symbol, date
        """,
        
        'moving_averages': """
            CREATE OR REPLACE VIEW moving_averages AS
            SELECT 
                date,
                symbol,
                close,
                volume,
                -- Simple moving averages
                AVG(close) OVER (
                    PARTITION BY symbol 
                    ORDER BY date 
                    ROWS BETWEEN 9 PRECEDING AND CURRENT ROW
                ) as sma_10,
                AVG(close) OVER (
                    PARTITION BY symbol 
                    ORDER BY date 
                    ROWS BETWEEN 19 PRECEDING AND CURRENT ROW
                ) as sma_20,
                AVG(close) OVER (
                    PARTITION BY symbol 
                    ORDER BY date 
                    ROWS BETWEEN 49 PRECEDING AND CURRENT ROW
                ) as sma_50,
                -- Volume weighted average price (20-day)
                SUM(close * volume) OVER (
                    PARTITION BY symbol 
                    ORDER BY date 
                    ROWS BETWEEN 19 PRECEDING AND CURRENT ROW
                ) / NULLIF(SUM(volume) OVER (
                    PARTITION BY symbol 
                    ORDER BY date 
                    ROWS BETWEEN 19 PRECEDING AND CURRENT ROW
                ), 0) as vwap_20
            FROM stock_prices
            ORDER BY symbol, date
        """,
        
        'volatility_metrics': """
            CREATE OR REPLACE VIEW volatility_metrics AS
            WITH returns AS (
                SELECT * FROM daily_returns WHERE log_return IS NOT NULL
            )
            SELECT 
                symbol,
                date,
                log_return,
                -- Rolling volatility (20-day)
                STDDEV(log_return) OVER (
                    PARTITION BY symbol 
                    ORDER BY date 
                    ROWS BETWEEN 19 PRECEDING AND CURRENT ROW
                ) * SQRT(252) as volatility_20d,
                -- Rolling volatility (60-day)
                STDDEV(log_return) OVER (
                    PARTITION BY symbol 
                    ORDER BY date 
                    ROWS BETWEEN 59 PRECEDING AND CURRENT ROW
                ) * SQRT(252) as volatility_60d
            FROM returns
        """,
        
        'market_summary': """
            CREATE OR REPLACE VIEW market_summary AS
            WITH latest_data AS (
                SELECT DISTINCT ON (symbol) 
                    symbol,
                    date,
                    close,
                    volume
                FROM stock_prices
                ORDER BY symbol, date DESC
            ),
            period_stats AS (
                SELECT 
                    symbol,
                    AVG(close) as avg_price_30d,
                    MIN(close) as min_price_30d,
                    MAX(close) as max_price_30d,
                    SUM(volume) as total_volume_30d
                FROM stock_prices
                WHERE date >= (SELECT MAX(date) FROM stock_prices) - INTERVAL '30 days'
                GROUP BY symbol
            ),
            return_stats AS (
                SELECT 
                    symbol,
                    AVG(log_return) * 252 as annual_return,
                    STDDEV(log_return) * SQRT(252) as annual_volatility
                FROM daily_returns
                WHERE log_return IS NOT NULL
                  AND date >= (SELECT MAX(date) FROM stock_prices) - INTERVAL '252 days'
                GROUP BY symbol
            )
            SELECT 
                l.symbol,
                l.date as last_update,
                l.close as current_price,
                p.avg_price_30d,
                CASE 
                    WHEN p.avg_price_30d > 0 
                    THEN (l.close - p.avg_price_30d) / p.avg_price_30d 
                    ELSE 0 
                END as pct_from_avg_30d,
                p.min_price_30d,
                p.max_price_30d,
                r.annual_return,
                r.annual_volatility,
                CASE 
                    WHEN r.annual_volatility > 0 
                    THEN r.annual_return / r.annual_volatility 
                    ELSE 0 
                END as sharpe_ratio
            FROM latest_data l
            LEFT JOIN period_stats p ON l.symbol = p.symbol
            LEFT JOIN return_stats r ON l.symbol = r.symbol
        """
    }
    
    # Create each view
    for view_name, view_sql in views.items():
        try:
            conn = psycopg2.connect(**config.get_connection_params())
            conn.autocommit = True
            cur = conn.cursor()
            
            print(f"Creating view: {view_name}...", end='')
            cur.execute(view_sql)
            results['views']['created'].append(view_name)
            print(" ✅")
            
            cur.close()
            conn.close()
            
        except Exception as e:
            print(f" ❌ Error: {str(e)[:50]}...")
            results['views']['failed'].append((view_name, str(e)))
            if conn:
                conn.close()
    
    # PART 2: CREATE INDEXES
    print("\n🔧 PART 2: Creating Performance Indexes")
    print("-" * 40)
    
    indexes = [
        ("idx_stock_prices_symbol", "stock_prices", "(symbol)"),
        ("idx_stock_prices_date", "stock_prices", "(date)"),
        ("idx_stock_prices_symbol_date", "stock_prices", "(symbol, date)"),
        ("idx_crypto_prices_symbol", "crypto_prices", "(symbol)"),
        ("idx_crypto_prices_timestamp", "crypto_prices", "(timestamp)"),
        ("idx_portfolio_holdings_date", "portfolio_holdings", "(date)"),
        ("idx_portfolio_holdings_portfolio_id", "portfolio_holdings", "(portfolio_id)"),
        ("idx_economic_indicators_indicator", "economic_indicators", "(indicator)"),
        ("idx_economic_indicators_date", "economic_indicators", "(date)"),
        ("idx_customers_risk_segment", "customers", "(risk_segment)"),
        ("idx_customers_credit_score", "customers", "(credit_score)")
    ]
    
    for idx_name, table, columns in indexes:
        try:
            conn = psycopg2.connect(**config.get_connection_params())
            conn.autocommit = True
            cur = conn.cursor()
            
            # Check if index exists
            cur.execute("""
                SELECT EXISTS (
                    SELECT 1 FROM pg_indexes 
                    WHERE schemaname = 'public' AND indexname = %s
                )
            """, (idx_name,))
            
            if cur.fetchone()[0]:
                print(f"{idx_name}: Already exists ✓")
            else:
                print(f"Creating {idx_name}...", end='')
                cur.execute(f"CREATE INDEX {idx_name} ON {table} {columns}")
                results['indexes']['created'].append(idx_name)
                print(" ✅")
            
            cur.close()
            conn.close()
            
        except Exception as e:
            print(f" ❌ Error: {e}")
            results['indexes']['failed'].append((idx_name, str(e)))
            if conn:
                conn.close()
    
    # PART 3: CREATE CORRELATION CALCULATIONS
    print("\n📊 PART 3: Setting Up Correlation Analysis")
    print("-" * 40)
    
    try:
        conn = psycopg2.connect(**config.get_connection_params())
        conn.autocommit = True
        cur = conn.cursor()
        
        # Create correlation calculation table
        print("Creating correlation storage table...", end='')
        cur.execute("""
            CREATE TABLE IF NOT EXISTS stock_correlations (
                symbol1 VARCHAR(10),
                symbol2 VARCHAR(10),
                correlation NUMERIC(10,6),
                observations INTEGER,
                period_start DATE,
                period_end DATE,
                calculated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                PRIMARY KEY (symbol1, symbol2)
            )
        """)
        print(" ✅")
        
        # Calculate correlations directly
        print("Calculating correlations (this may take a moment)...")
        
        # Get list of symbols
        cur.execute("SELECT DISTINCT symbol FROM stock_prices ORDER BY symbol")
        symbols = [row[0] for row in cur.fetchall()]
        
        # Clear old correlations
        cur.execute("TRUNCATE stock_correlations")
        
        # Calculate correlations for each pair
        correlation_count = 0
        total_pairs = len(symbols) * (len(symbols) - 1) // 2
        
        for i, symbol1 in enumerate(symbols):
            for j, symbol2 in enumerate(symbols[i+1:], i+1):
                # Progress indicator
                if correlation_count % 10 == 0:
                    progress = (correlation_count / total_pairs) * 100
                    print(f"\r  Progress: {progress:.1f}% ({correlation_count}/{total_pairs})", end='')
                
                cur.execute("""
                    WITH paired_returns AS (
                        SELECT 
                            r1.date,
                            r1.log_return as return1,
                            r2.log_return as return2
                        FROM daily_returns r1
                        JOIN daily_returns r2 ON r1.date = r2.date
                        WHERE r1.symbol = %s 
                          AND r2.symbol = %s
                          AND r1.log_return IS NOT NULL
                          AND r2.log_return IS NOT NULL
                    )
                    SELECT 
                        CORR(return1, return2) as correlation,
                        COUNT(*) as observations,
                        MIN(date) as period_start,
                        MAX(date) as period_end
                    FROM paired_returns
                    HAVING COUNT(*) >= 30
                """, (symbol1, symbol2))
                
                result = cur.fetchone()
                if result and result[0] is not None:
                    cur.execute("""
                        INSERT INTO stock_correlations 
                        (symbol1, symbol2, correlation, observations, period_start, period_end)
                        VALUES (%s, %s, %s, %s, %s, %s)
                    """, (symbol1, symbol2, result[0], result[1], result[2], result[3]))
                    correlation_count += 1
        
        print(f"\n✅ Calculated {correlation_count} correlations")
        results['correlations']['count'] = correlation_count
        results['correlations']['status'] = 'success'
        
        # Create correlation view
        print("Creating correlation view...", end='')
        cur.execute("""
            CREATE OR REPLACE VIEW correlation_matrix AS
            SELECT symbol1, symbol2, correlation, observations
            FROM stock_correlations
            UNION ALL
            SELECT symbol2, symbol1, correlation, observations
            FROM stock_correlations
            UNION ALL
            SELECT DISTINCT symbol, symbol, 1.0::numeric, COUNT(*)::integer
            FROM stock_prices
            GROUP BY symbol
        """)
        print(" ✅")
        
        cur.close()
        conn.close()
        
    except Exception as e:
        print(f"\n❌ Correlation calculation error: {e}")
        results['correlations']['status'] = 'failed'
        if conn:
            conn.close()
    
    # PART 4: TEST THE INFRASTRUCTURE
    print("\n🔍 PART 4: Testing Analysis Infrastructure")
    print("-" * 40)
    
    test_queries = {
        "Market Summary (Top Movers)": """
            SELECT symbol, current_price, 
                   ROUND(CAST(pct_from_avg_30d * 100 AS numeric), 2) as pct_change_30d
            FROM market_summary
            WHERE current_price IS NOT NULL
            ORDER BY ABS(pct_from_avg_30d) DESC NULLS LAST
            LIMIT 5
        """,
        
        "Recent Volatility Leaders": """
            SELECT DISTINCT symbol, 
                   ROUND(CAST(volatility_20d * 100 AS numeric), 1) as volatility_pct
            FROM volatility_metrics
            WHERE date = (SELECT MAX(date) FROM volatility_metrics)
              AND volatility_20d IS NOT NULL
            ORDER BY volatility_20d DESC
            LIMIT 5
        """,
        
        "Highest Correlations": """
            SELECT symbol1, symbol2, 
                   ROUND(correlation, 3) as corr,
                   observations as obs
            FROM stock_correlations
            WHERE correlation < 0.999
            ORDER BY correlation DESC
            LIMIT 5
        """,
        
        "Moving Average Signals": """
            SELECT symbol,
                   ROUND(CAST(close AS numeric), 2) as price,
                   ROUND(CAST(sma_20 AS numeric), 2) as sma20,
                   CASE 
                       WHEN close > sma_20 THEN 'Above'
                       WHEN close < sma_20 THEN 'Below'
                       ELSE 'At'
                   END as signal
            FROM moving_averages
            WHERE date = (SELECT MAX(date) FROM moving_averages)
              AND sma_20 IS NOT NULL
            ORDER BY symbol
            LIMIT 5
        """
    }
    
    for query_name, query_sql in test_queries.items():
        print(f"\n{query_name}:")
        try:
            conn = psycopg2.connect(**config.get_connection_params())
            cur = conn.cursor()
            
            cur.execute(query_sql)
            results_data = cur.fetchall()
            
            if results_data:
                # Get column names
                columns = [desc[0] for desc in cur.description]
                
                # Print header
                header = " | ".join(f"{col[:12]:^12}" for col in columns)
                print(f"  {header}")
                print(f"  {'-' * len(header)}")
                
                # Print data
                for row in results_data:
                    formatted_row = []
                    for val in row:
                        if isinstance(val, (int, float)):
                            formatted_row.append(f"{val:^12.2f}")
                        else:
                            formatted_row.append(f"{str(val)[:12]:^12}")
                    print(f"  {' | '.join(formatted_row)}")
            else:
                print("  No data available")
            
            cur.close()
            conn.close()
            
        except Exception as e:
            print(f"  Query failed: {e}")
    
    # PART 5: CREATE USEFUL HELPER FUNCTIONS
    print("\n🛠️ PART 5: Creating Helper Functions")
    print("-" * 40)
    
    try:
        conn = psycopg2.connect(**config.get_connection_params())
        conn.autocommit = True
        cur = conn.cursor()
        
        # Create a function to get correlations for a specific symbol
        print("Creating correlation lookup function...", end='')
        cur.execute("""
            CREATE OR REPLACE FUNCTION get_correlations_for_symbol(target_symbol VARCHAR)
            RETURNS TABLE(
                symbol VARCHAR,
                correlation NUMERIC,
                observations INTEGER
            ) AS $$
            BEGIN
                RETURN QUERY
                SELECT 
                    CASE 
                        WHEN symbol1 = target_symbol THEN symbol2
                        ELSE symbol1
                    END as symbol,
                    correlation,
                    observations
                FROM stock_correlations
                WHERE symbol1 = target_symbol OR symbol2 = target_symbol
                ORDER BY correlation DESC;
            END;
            $$ LANGUAGE plpgsql;
        """)
        print(" ✅")
        
        # Create a function to calculate returns between two dates
        print("Creating returns calculation function...", end='')
        cur.execute("""
            CREATE OR REPLACE FUNCTION calculate_returns(
                start_date DATE,
                end_date DATE
            )
            RETURNS TABLE(
                symbol VARCHAR,
                start_price NUMERIC,
                end_price NUMERIC,
                total_return NUMERIC,
                annualized_return NUMERIC
            ) AS $$
            BEGIN
                RETURN QUERY
                WITH price_data AS (
                    SELECT DISTINCT ON (symbol)
                        symbol,
                        FIRST_VALUE(close) OVER (PARTITION BY symbol ORDER BY date) as start_price,
                        LAST_VALUE(close) OVER (PARTITION BY symbol ORDER BY date 
                            RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as end_price
                    FROM stock_prices
                    WHERE date BETWEEN start_date AND end_date
                )
                SELECT 
                    pd.symbol,
                    pd.start_price,
                    pd.end_price,
                    (pd.end_price - pd.start_price) / pd.start_price as total_return,
                    POWER((pd.end_price / pd.start_price), 
                          365.0 / (end_date - start_date)::numeric) - 1 as annualized_return
                FROM price_data pd;
            END;
            $$ LANGUAGE plpgsql;
        """)
        print(" ✅")
        
        cur.close()
        conn.close()
        
    except Exception as e:
        print(f"\n❌ Helper function error: {e}")
    
    # FINAL SUMMARY
    print("\n" + "=" * 60)
    print("📊 ANALYSIS INFRASTRUCTURE SUMMARY")
    print("=" * 60)
    
    print(f"\nViews:")
    print(f"  ✅ Created: {len(results['views']['created'])}")
    print(f"  ❌ Failed: {len(results['views']['failed'])}")
    if results['views']['created']:
        print(f"  Available views: {', '.join(results['views']['created'])}")
    
    print(f"\nIndexes:")
    print(f"  ✅ Created/Verified: {len(results['indexes']['created']) + (len(indexes) - len(results['indexes']['failed']) - len(results['indexes']['created']))}")
    print(f"  ❌ Failed: {len(results['indexes']['failed'])}")
    
    print(f"\nCorrelations:")
    print(f"  Status: {results['correlations']['status']}")
    print(f"  Calculated: {results['correlations']['count']} pairs")
    
    print("\n📚 Available Analysis Tools:")
    print("-" * 40)
    print("Views:")
    for view in ['daily_returns', 'moving_averages', 'volatility_metrics', 'market_summary', 'correlation_matrix']:
        if view in results['views']['created'] or view == 'correlation_matrix':
            print(f"  ✓ {view}")
    
    print("\nHelper Functions:")
    print("  ✓ get_correlations_for_symbol(symbol)")
    print("  ✓ calculate_returns(start_date, end_date)")
    
    print("\n📝 Example Usage:")
    print("-" * 40)
    print("-- Get all correlations for Apple")
    print("SELECT * FROM get_correlations_for_symbol('AAPL');")
    print("")
    print("-- Calculate returns for 2023")
    print("SELECT * FROM calculate_returns('2023-01-01', '2023-12-31');")
    print("")
    print("-- Find stocks with positive momentum")
    print("SELECT symbol, close, sma_20 ")
    print("FROM moving_averages ")
    print("WHERE date = (SELECT MAX(date) FROM moving_averages)")
    print("  AND close > sma_20 ")
    print("  AND close > sma_50;")
    
    print("\n✅ Analysis infrastructure setup complete!")
    print("🎉 You're now ready for advanced financial analysis!")
    
    return results

# Run the fixed infrastructure setup
print("🚀 Starting infrastructure setup...\n")
infrastructure_results = create_complete_analysis_infrastructure_fixed(db_config)

# Quick validation of what was created
print("\n🔍 Quick Validation:")
try:
    conn = psycopg2.connect(**db_config.get_connection_params())
    cur = conn.cursor()
    
    # Count views
    cur.execute("""
        SELECT COUNT(*) FROM pg_views 
        WHERE schemaname = 'public'
    """)
    view_count = cur.fetchone()[0]
    print(f"  Total views in database: {view_count}")
    
    # Count indexes
    cur.execute("""
        SELECT COUNT(*) FROM pg_indexes 
        WHERE schemaname = 'public'
    """)
    index_count = cur.fetchone()[0]
    print(f"  Total indexes in database: {index_count}")
    
    # Count correlations
    cur.execute("SELECT COUNT(*) FROM stock_correlations")
    corr_count = cur.fetchone()[0]
    print(f"  Total correlations calculated: {corr_count}")
    
    cur.close()
    conn.close()
    
except Exception as e:
    print(f"  Validation error: {e}")

print("\n💾 Infrastructure setup complete! Ready for Week 2.")

🚀 Starting infrastructure setup...

🏗️ Creating Complete Analysis Infrastructure

🧹 PART 0: Cleaning Up Existing Objects
----------------------------------------
⚠️ Cleanup note: "correlation_matrix" is not a view
HINT:  Use DROP MATERIALIZED VIEW to remove a materialized view.


📊 PART 1: Creating Analysis Views
----------------------------------------
Creating view: daily_returns... ✅
Creating view: moving_averages... ❌ Error: cannot change name of view column "sma_10" to "vol...
Creating view: volatility_metrics... ✅
Creating view: market_summary... ✅

🔧 PART 2: Creating Performance Indexes
----------------------------------------
idx_stock_prices_symbol: Already exists ✓
idx_stock_prices_date: Already exists ✓
idx_stock_prices_symbol_date: Already exists ✓
idx_crypto_prices_symbol: Already exists ✓
idx_crypto_prices_timestamp: Already exists ✓
idx_portfolio_holdings_date: Already exists ✓
idx_portfolio_holdings_portfolio_id: Already exists ✓
idx_economic_indicators_indicator: Alrea

SQL Practice Exercises for Financial Analysis
=====================================================

Now let's practice SQL with real financial queries.
These exercises progress from basic to advanced.

Try to solve each one before looking at the solution!


In [36]:
def generate_sql_exercises():
    """
    Generate a set of SQL exercises with solutions.
    """
    
    exercises = [
        {
            'level': 'Beginner',
            'title': 'Find Recent Prices',
            'question': 'Get the last 10 closing prices for Apple (AAPL)',
            'hint': 'Use WHERE for filtering and ORDER BY with LIMIT',
            'solution': """
SELECT date, close
FROM stock_prices
WHERE symbol = 'AAPL'
ORDER BY date DESC
LIMIT 10;
            """
        },
        {
            'level': 'Beginner',
            'title': 'Average Volume',
            'question': 'Calculate the average daily trading volume for each stock',
            'hint': 'Use GROUP BY and AVG()',
            'solution': """
SELECT 
    symbol,
    AVG(volume) as avg_daily_volume,
    MIN(volume) as min_volume,
    MAX(volume) as max_volume
FROM stock_prices
GROUP BY symbol
ORDER BY avg_daily_volume DESC;
            """
        },
        {
            'level': 'Intermediate',
            'title': 'Monthly Performance',
            'question': 'Calculate monthly returns for each stock in 2023',
            'hint': 'Use DATE_TRUNC() and window functions',
            'solution': """
WITH monthly_prices AS (
    SELECT 
        symbol,
        DATE_TRUNC('month', date) as month,
        FIRST_VALUE(open) OVER (PARTITION BY symbol, DATE_TRUNC('month', date) ORDER BY date) as month_open,
        LAST_VALUE(close) OVER (PARTITION BY symbol, DATE_TRUNC('month', date) ORDER BY date 
                                RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as month_close
    FROM stock_prices
    WHERE EXTRACT(YEAR FROM date) = 2023
)
SELECT DISTINCT
    symbol,
    month,
    month_open,
    month_close,
    (month_close - month_open) / month_open * 100 as monthly_return_pct
FROM monthly_prices
ORDER BY symbol, month;
            """
        },
        {
            'level': 'Intermediate',
            'title': 'Volatility Ranking',
            'question': 'Rank stocks by their 30-day volatility',
            'hint': 'Calculate standard deviation of returns',
            'solution': """
WITH daily_returns AS (
    SELECT 
        symbol,
        date,
        (close - LAG(close) OVER (PARTITION BY symbol ORDER BY date)) / 
         LAG(close) OVER (PARTITION BY symbol ORDER BY date) as return
    FROM stock_prices
    WHERE date >= CURRENT_DATE - INTERVAL '30 days'
)
SELECT 
    symbol,
    STDDEV(return) * SQRT(252) as annualized_volatility,
    COUNT(*) as trading_days,
    RANK() OVER (ORDER BY STDDEV(return) DESC) as volatility_rank
FROM daily_returns
WHERE return IS NOT NULL
GROUP BY symbol
HAVING COUNT(*) > 20  -- Ensure enough data points
ORDER BY annualized_volatility DESC;
            """
        },
        {
            'level': 'Advanced',
            'title': 'Portfolio Correlation',
            'question': 'Find which stocks move together (correlation > 0.7)',
            'hint': 'Self-join daily returns and use CORR() function',
            'solution': """
WITH returns AS (
    SELECT 
        date,
        symbol,
        (close - LAG(close) OVER (PARTITION BY symbol ORDER BY date)) / 
         LAG(close) OVER (PARTITION BY symbol ORDER BY date) as return
    FROM stock_prices
)
SELECT 
    r1.symbol as symbol1,
    r2.symbol as symbol2,
    CORR(r1.return, r2.return) as correlation,
    COUNT(*) as observations
FROM returns r1
JOIN returns r2 ON r1.date = r2.date
WHERE r1.symbol < r2.symbol  -- Avoid duplicates
  AND r1.return IS NOT NULL
  AND r2.return IS NOT NULL
GROUP BY r1.symbol, r2.symbol
HAVING CORR(r1.return, r2.return) > 0.7
   AND COUNT(*) > 100
ORDER BY correlation DESC;
            """
        },
        {
            'level': 'Advanced',
            'title': 'Economic Impact Analysis',
            'question': 'Analyze how stock returns correlate with GDP growth',
            'hint': 'JOIN stock and economic data, calculate correlations',
            'solution': """
WITH monthly_stock_returns AS (
    SELECT 
        DATE_TRUNC('month', date) as month,
        symbol,
        (MAX(close) - MIN(open)) / MIN(open) as monthly_return
    FROM stock_prices
    GROUP BY DATE_TRUNC('month', date), symbol
),
gdp_data AS (
    SELECT 
        date,
        value as gdp_growth
    FROM economic_indicators
    WHERE indicator = 'GDP_GROWTH'
)
SELECT 
    s.symbol,
    CORR(s.monthly_return, g.gdp_growth) as return_gdp_correlation,
    COUNT(*) as observations,
    AVG(s.monthly_return) * 12 as avg_annual_return,
    AVG(g.gdp_growth) as avg_gdp_growth
FROM monthly_stock_returns s
JOIN gdp_data g ON DATE_TRUNC('month', g.date) = s.month
GROUP BY s.symbol
HAVING COUNT(*) > 12  -- At least 1 year of data
ORDER BY return_gdp_correlation DESC;
            """
        }
    ]
    
    # Create exercise notebook
    print("📝 SQL Exercise Notebook")
    print("=" * 60)
    
    for i, exercise in enumerate(exercises, 1):
        print(f"\n{'🟢' if exercise['level'] == 'Beginner' else '🟡' if exercise['level'] == 'Intermediate' else '🔴'} Exercise {i}: {exercise['title']} ({exercise['level']})")
        print("-" * 40)
        print(f"Question: {exercise['question']}")
        print(f"Hint: {exercise['hint']}")
        print("\nTry writing your query here first:")
        print("```sql")
        print("-- Your solution\n\n")
        print("```")
        
        show_solution = input("\nPress Enter to see solution (or 's' to skip): ")
        if show_solution.lower() != 's':
            print("\nSolution:")
            print("```sql")
            print(exercise['solution'].strip())
            print("```")
    
    return exercises

# Run SQL exercises
exercises = generate_sql_exercises()

📝 SQL Exercise Notebook

🟢 Exercise 1: Find Recent Prices (Beginner)
----------------------------------------
Question: Get the last 10 closing prices for Apple (AAPL)
Hint: Use WHERE for filtering and ORDER BY with LIMIT

Try writing your query here first:
```sql
-- Your solution


```



Press Enter to see solution (or 's' to skip):  



Solution:
```sql
SELECT date, close
FROM stock_prices
WHERE symbol = 'AAPL'
ORDER BY date DESC
LIMIT 10;
```

🟢 Exercise 2: Average Volume (Beginner)
----------------------------------------
Question: Calculate the average daily trading volume for each stock
Hint: Use GROUP BY and AVG()

Try writing your query here first:
```sql
-- Your solution


```



Press Enter to see solution (or 's' to skip):  



Solution:
```sql
SELECT 
    symbol,
    AVG(volume) as avg_daily_volume,
    MIN(volume) as min_volume,
    MAX(volume) as max_volume
FROM stock_prices
GROUP BY symbol
ORDER BY avg_daily_volume DESC;
```

🟡 Exercise 3: Monthly Performance (Intermediate)
----------------------------------------
Question: Calculate monthly returns for each stock in 2023
Hint: Use DATE_TRUNC() and window functions

Try writing your query here first:
```sql
-- Your solution


```


...


Solution:
```sql
WITH daily_returns AS (
    SELECT 
        symbol,
        date,
        (close - LAG(close) OVER (PARTITION BY symbol ORDER BY date)) / 
         LAG(close) OVER (PARTITION BY symbol ORDER BY date) as return
    FROM stock_prices
    WHERE date >= CURRENT_DATE - INTERVAL '30 days'
)
SELECT 
    symbol,
    STDDEV(return) * SQRT(252) as annualized_volatility,
    COUNT(*) as trading_days,
    RANK() OVER (ORDER BY STDDEV(return) DESC) as volatility_rank
FROM daily_returns
WHERE return IS NOT NULL
GROUP BY symbol
HAVING COUNT(*) > 20  -- Ensure enough data points
ORDER BY annualized_volatility DESC;
```

🔴 Exercise 5: Portfolio Correlation (Advanced)
----------------------------------------
Question: Find which stocks move together (correlation > 0.7)
Hint: Self-join daily returns and use CORR() function

Try writing your query here first:
```sql
-- Your solution


```



Press Enter to see solution (or 's' to skip):  



Solution:
```sql
WITH returns AS (
    SELECT 
        date,
        symbol,
        (close - LAG(close) OVER (PARTITION BY symbol ORDER BY date)) / 
         LAG(close) OVER (PARTITION BY symbol ORDER BY date) as return
    FROM stock_prices
)
SELECT 
    r1.symbol as symbol1,
    r2.symbol as symbol2,
    CORR(r1.return, r2.return) as correlation,
    COUNT(*) as observations
FROM returns r1
JOIN returns r2 ON r1.date = r2.date
WHERE r1.symbol < r2.symbol  -- Avoid duplicates
  AND r1.return IS NOT NULL
  AND r2.return IS NOT NULL
GROUP BY r1.symbol, r2.symbol
HAVING CORR(r1.return, r2.return) > 0.7
   AND COUNT(*) > 100
ORDER BY correlation DESC;
```

🔴 Exercise 6: Economic Impact Analysis (Advanced)
----------------------------------------
Question: Analyze how stock returns correlate with GDP growth
Hint: JOIN stock and economic data, calculate correlations

Try writing your query here first:
```sql
-- Your solution


```



Press Enter to see solution (or 's' to skip):  



Solution:
```sql
WITH monthly_stock_returns AS (
    SELECT 
        DATE_TRUNC('month', date) as month,
        symbol,
        (MAX(close) - MIN(open)) / MIN(open) as monthly_return
    FROM stock_prices
    GROUP BY DATE_TRUNC('month', date), symbol
),
gdp_data AS (
    SELECT 
        date,
        value as gdp_growth
    FROM economic_indicators
    WHERE indicator = 'GDP_GROWTH'
)
SELECT 
    s.symbol,
    CORR(s.monthly_return, g.gdp_growth) as return_gdp_correlation,
    COUNT(*) as observations,
    AVG(s.monthly_return) * 12 as avg_annual_return,
    AVG(g.gdp_growth) as avg_gdp_growth
FROM monthly_stock_returns s
JOIN gdp_data g ON DATE_TRUNC('month', g.date) = s.month
GROUP BY s.symbol
HAVING COUNT(*) > 12  -- At least 1 year of data
ORDER BY return_gdp_correlation DESC;
```


Week 1 Complete - Integration and Next Steps
====================================================

Congratulations! You've successfully:
✅ Set up PostgreSQL database
✅ Created proper table schemas
✅ Imported all financial datasets
✅ Created analysis views
✅ Validated data quality
✅ Practiced SQL queries

For your 10-minute presentation, prepare:
1. **Database Schema Diagram**: Show your understanding of table relationships
2. **Interesting Query**: One SQL query that reveals a pattern
3. **Data Quality Issue**: Any problem you found and how to fix it
4. **Project Vision**: How this data supports your charter goals

Let's wrap up and prepare for Week 2.

Your data pipeline is ready! Next week we'll:
1. Clean and validate data using pandas/polars
2. Handle missing values and outliers
3. Create derived features
4. Build visualizations
5. Generate insights
